In [1]:
%load_ext autoreload
%autoreload 2
from dotenv import load_dotenv
import os
import sys
import logging
from openpyxl import load_workbook
from datetime import datetime, date
from datetime import time as dtTime
import pandas as pd
import threading
import numpy as np
from pathos.multiprocessing import ProcessingPool as Pool
from concurrent.futures import ProcessPoolExecutor, as_completed
import concurrent.futures
from trade.assets.Stock import Stock
from trade.helpers.helper import generate_option_tick_new
from trade.assets.rates import get_risk_free_rate_helper
from trade.helpers.helper import IV_handler, time_distance_helper, binomial_implied_vol, wait_for_response, HOLIDAY_SET,enforce_allowed_models,optionPV_helper
from trade.helpers.helper import extract_numeric_value, change_to_last_busday, parse_option_tick, CustomCache
from trade.helpers.helper import optionPV_helper
from trade.helpers.exception import IncorrectExecutionError
from trade.helpers.Logging import setup_logger
from trade.assets.Calculate import Calculate
from trade.helpers.Context import Context
from dbase.DataAPI.ThetaData import (retrieve_ohlc, 
                                     retrieve_quote_rt, 
                                     retrieve_eod_ohlc, 
                                     resample, 
                                     retrieve_quote, 
                                     enforce_bus_hours,
                                     retrieve_bulk_eod,
                                     retrieve_openInterest,
                                     retrieve_chain_bulk,
                                     list_contracts,
                                     retrieve_bulk_open_interest
                                     )
from trade.helpers.threads import runThreads
from trade.helpers.pools import runProcesses, parallel_apply
from dbase.DataAPI.Organizers import generate_optionData_to_save, Calc_Risks
from dbase.database.SQLHelpers import store_SQL_data_Insert_Ignore, query_database, dynamic_batch_update
from trade.helpers.decorators import log_error, log_error_with_stack, log_time
from trade.helpers.types import OptionModelAttributes
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import BDay
from dbase.database.SQLHelpers import DatabaseAdapter
from trade.models.VolSurface import fit_svi_model
from trade.models.utils import resolve_missing_vol
from threading import Thread
from dbase.utils import add_eod_timestamp, bus_range, enforce_bus_hours, default_timestamp
from trade.helpers.pools import runProcesses
from pathos.multiprocessing import ProcessingPool as Pool
from copy import deepcopy
pd.options.display.max_columns = 100
from abc import ABC, abstractmethod
logger = setup_logger('test_datamanager')
time_logger = setup_logger('time_logger_test_dm')
POOL_ENABLED = bool(os.environ.get('POOL_ENABLED'))
CENTRAL_SAVE_THREAD = {}


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-05-19 02:34:33 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantTools/logs
Using Proxy URL: http://18.232.166.224:5500/thetadata


In [2]:

## Format of tables is: database_name.table_name
TABLES = {
    'eod':{
        'attribution': 'securities_master.attribution_eod',
        'spot': 'securities_master.temp_options_eod_new',
        'vol': 'securities_master.temp_options_eod_new',
        'greeks': 'securities_master.temp_options_eod_new',
        'chain': 'vol_surface.option_chain'
    },
    'intra':{
        'attribution': 'securities_master.attribution_intra',
        'spot': 'securities_master.temp_options_intra_new',
        'vol': 'securities_master.temp_options_intra_new',
        'greeks': 'securities_master.temp_options_intra_new',
    }
}

import json
with open(f"{os.environ['WORK_DIR']}/pricingConfig.json") as f:
    PRICING_CONFIG = json.load(f)
PRICING_CONFIG

{'INTRADAY_AGG': '5m',
 'MARKET_OPEN_TIME': '09:30',
 'MARKET_CLOSE_TIME': '16:00',
 'AVAILABLE_PRICING_MODELS': ['bs', 'binomial', 'mc'],
 'AVAILABLE_INTERVALS': ['h', 'd', 'w', 'q', 'y', 'M', 'm'],
 'AVAILABLE_GREEKS': ['vega',
  'vanna',
  'volga',
  'delta',
  'gamma',
  'theta',
  'rho'],
 'UPPER_BOUND_MONEYNESS': 1.2,
 'LOWER_BOUND_MONEYNESS': 0.8}

In [52]:
from pathlib import Path
DB_CACHE_LOCATION = Path(os.environ['WORK_DIR']) / '.cache' 
DB_CACHE = CustomCache(DB_CACHE_LOCATION, fname = 'dm_cache', clear_on_exit=False, expire_days = 180)

In [57]:
with Path(DB_CACHE.register_location).open() as f:
    register = json.load(f)
register


{'/Users/chiemelienwanisobi/cloned_repos/QuantTools/.cache/dm_cache': '2025-11-15',
 '/Users/chiemelienwanisobi/cloned_repos/QuantTools/.riskmanager_cache/chain/bBJszpyN': '2025-07-03',
 '/Users/chiemelienwanisobi/cloned_repos/QuantTools/.riskmanager_cache/close/XzNozWyc': '2025-07-03',
 '/Users/chiemelienwanisobi/cloned_repos/QuantTools/.riskmanager_cache/oi/MWdKMrH7': '2025-07-03',
 '/Users/chiemelienwanisobi/cloned_repos/QuantTools/.riskmanager_cache/spot/SKb4khEe': '2025-07-03',
 '/Users/chiemelienwanisobi/cloned_repos/QuantTools/.riskmanager_cache/order/DiWYKWsv': '2025-05-26',
 '/Users/chiemelienwanisobi/cloned_repos/QuantTools/.riskmanager_cache/rm_spot_timeseries/cmeNf34E': '2025-05-26',
 '/Users/chiemelienwanisobi/cloned_repos/QuantTools/.riskmanager_cache/rm_chain_spot_timeseries/VPQjaKG3': '2025-05-26',
 '/Users/chiemelienwanisobi/cloned_repos/QuantTools/.riskmanager_cache/rm_processed_option_data/baAM7NUS': '2025-05-26',
 '/Users/chiemelienwanisobi/cloned_repos/QuantTools/.

In [11]:
from EventDriven.riskmanager.utils import assemble_bulk_data_request
from module_test.raw_code.DataManagers.DataManagers import BulkOptionDataManager, OptionDataManager
from module_test.raw_code.DataManagers.Requests import ChainDataRequest
from trade.helpers.helper import check_all_days_available, check_missing_dates
#SBUX20240621C110, SBUX20240621C155
manager = BulkOptionDataManager(symbol = 'SBUX', exp ='20240621' )
bulk_request = assemble_bulk_data_request(
    self = manager,
    start = '2023-03-07',
    end = '2023-12-31',
    type_ = 'spot',
    strikes_right = [(110.0, 'C'), (115.0, 'C'), (130.0, 'C')]

)
bulk_request

In [21]:
single_manager = OptionDataManager('SBUX', exp = '20240621', strike = 110.0, right = 'C')
single_request = single_manager.get_timeseries(
    start = '2023-03-07',
    end = '2023-03-14',
    type_ = 'spot',
)

bulk_manager = BulkOptionDataManager('SBUX', exp = '20240621')
bulk_request = bulk_manager.get_timeseries(
    start = '2023-03-07',
    end = '2023-03-14',
    type_ = 'spot',
    strikes_right = [(110.0, 'C'), (115.0, 'C'), (130.0, 'C')]
)

Using Cache
Using Cache
2025-05-19 02:39:56 dbase.DataAPI.ThetaData CRITICAL: `datetime_col_name` not provided for multi index resample, setting to `Datetime`


In [49]:
import json
registry = f'{os.environ["WORK_DIR"]}/trade/helpers/clear_dirs.json'
with open(registry, 'r') as f:
    json_file = json.load(f)
print(json_file)
with open(registry, 'w') as f:
    loc = str(DB_CACHE.dir)
    json_file.update({loc: datetime.now().strftime('%Y-%m-%d')})
    json.dump(json_file, f, default=str)

{'eerrrrr': '2025-05-19'}


In [34]:
DB_CACHE.dir

PosixPath('/Users/chiemelienwanisobi/cloned_repos/QuantTools/.cache/dm_cache')

In [15]:
bulk_request.post_processed_data

Open  High   Low  Close  Midpoint  Volume  \
Optiontick       Datetime                                                 
SBUX20240621C110 2023-03-07   0.00   0.0  0.00    0.0    11.325       0   
                 2023-03-08   0.00   0.0  0.00    0.0    10.925       0   
                 2023-03-09   0.00   0.0  0.00    0.0     9.800       0   
                 2023-03-10  10.10  10.1  9.70    9.7     9.275      26   
                 2023-03-13   8.80   8.8  8.80    8.8     8.775       3   
...                            ...   ...   ...    ...       ...     ...   
SBUX20240621C130 2023-03-08   0.00   0.0  0.00    0.0     4.475       0   
                 2023-03-09   4.07   4.1  4.07    4.1     3.950       9   
                 2023-03-10   0.00   0.0  0.00    0.0     3.675       0   
                 2023-03-13   0.00   0.0  0.00    0.0     3.425       0   
                 2023-03-14   0.00   0.0  0.00    0.0     3.400       0   

                             Openinterest  
Optiontick       Datetime                  
SBUX20240621C110 2023-03-07             0  
                 2023-03-08             0  
                 2023-03-09             0  
                 2023-03-10             0  
                 2023-03-13            19  
...                                   ...  
SBUX20240621C130 2023-03-08             0  
                 2023-03-09             0  
                 2023-03-10             9  
                 2023-03-13             9  
                 2023-03-14             9  

[226 rows x 7 columns]

In [247]:
single_request.end_date ='2023-12-31'


In [165]:
chain_request = ChainDataRequest(
    'JPM',
    date = '2024-09-18',
    table_name = 'option_chain',
    db_name = 'vol_surface'
)
chain_request.db_name, chain_request.table_name

('vol_surface', 'option_chain')

In [257]:
DB_CACHE['securities_master.temp_options_eod_new'] = pd.DataFrame()
DB_CACHE['vol_surface.option_chain'] = pd.DataFrame()
# del DB_CACHE['securities_master.temp_options_eod_new']
# DB_CACHE

In [249]:
def query_cache(data_request, query_category):
    """
    Check if the data is available in the cache.

    Args:
        data_request (object): The data request object containing the parameters.
        query_category (str): The category of the query ('single', 'bulk', 'chain').
    Returns:
        bool: True if the data is available in the cache, False otherwise.

    Refer to:
        data_request.cache_data: The data retrieved from the cache.
        data_request.query_ticks: The ticks that are missing from the cache.
    """
    # 1) Get data from cache.
    cache_data = _check_cache(data_request, query_category)

    # 2) Check if it is complete or empty
    if query_category in ['single', 'bulk']:
        data_request.query_ticks, skip_db_query = timeseries_cache_validation(data_request, query_category)

    else:
        skip_db_query = not cache_data.empty
    
    data_request.cache_data = cache_data
    return skip_db_query

def _check_cache(data_request, query_category):
    """
    Check if the data is available in the cache.

    Args:
        data_request (object): The data request object containing the parameters.
        query_category (str): The category of the query ('single', 'bulk', 'chain').
    Returns:
        bool: True if the data is available in the cache, False otherwise.

    Refer to:
        data_request.cache_data: The data retrieved from the cache.
        data_request.query_ticks: The ticks that are missing from the cache.
    """
    global DB_CACHE
    tick = data_request.symbol

    if query_category in ['single', 'bulk']:
        exp = data_request.exp
        start, end = pd.to_datetime(data_request.start_date), pd.to_datetime(data_request.end_date)
    else:
        date = pd.to_datetime(data_request.date).date()

    # 1) What Table are we using?
    key = f"{data_request.db_name}.{data_request.table_name}"
    
    # 2) Handle Filtering based on query_category
    if query_category == 'single':
        opttick = [data_request.opttick]
        data_request.cache_opttick = opttick

    elif query_category == 'bulk':
        strikes_right = data_request.strikes
        opttick = [generate_option_tick_new(tick, x[1], exp, x[0]) for x in strikes_right]
        data_request.cache_opttick = opttick

    elif query_category == 'chain':
        pass
    
    else: raise ValueError("Unknown query_category. Recieved {query_category}".format(query_category))

    # 3) Get Data from db cache
    cache_data = DB_CACHE.get(key)
    if cache_data is None or cache_data.empty:
        return DB_CACHE.setdefault(key, pd.DataFrame())
    # 3.1) Filter based on type
    else:
        if query_category in ['single', 'bulk']:
            return cache_data[(cache_data.optiontick.isin(opttick)) & \
                              (cache_data.datetime >= start ) & \
                              (cache_data.datetime >= start )]
        else:
            return cache_data[(cache_data.ticker == tick) & \
                              (pd.DatetimeIndex(cache_data.build_date).date == date)]


def timeseries_cache_validation(data_request, query_category):
    """
    Validate the cache data for timeseries requests.
    Args:
        data_request (object): The data request object containing the parameters.
        query_category (str): The category of the query ('single', 'bulk', 'chain').
    Returns:
        tuple: A tuple containing the missing option ticks and a boolean indicating whether to skip the database query.
    """
    

    bulk_cache_data = _check_cache(data_request, query_category).copy()
    if bulk_cache_data.empty:
        return data_request.cache_opttick, False
    bulk_cache_data['Datetime'] = bulk_cache_data.datetime
    has_all_ticks = all(x in bulk_cache_data.optiontick.unique() for x in data_request.cache_opttick)
    missing_opttick = [x for x in data_request.cache_opttick if x not in bulk_cache_data.optiontick.unique()]
    bool_series = bulk_cache_data.groupby('optiontick').apply(check_all_days_available, _start = data_request.start_date, _end = data_request.end_date)
    is_available_option_tick_complete = bool_series.all()
    missing_opttick = missing_opttick + bool_series[bool_series==False].index.to_list()
    is_empty = bulk_cache_data.empty
    skip_db_query = has_all_ticks and is_available_option_tick_complete and not is_empty

    return missing_opttick, skip_db_query


In [250]:
query_cache(single_request, 'single')

True

In [252]:
single_request.query_ticks

[]

In [262]:
def init_query(**kwargs):

    """
    Initialize the query for the data request and save the data to the request
    """
    global DB_CACHE
    data_request = kwargs.get('data_request')
    db = kwargs.get('db', DatabaseAdapter())
    try:
        query_category = kwargs['query_category']
    except KeyError:
        raise KeyError("Query category not specified, expected one of: ['single', 'bulk', 'chain']")
    
    ## 1) Check if it already cached
    skip_db_query = query_cache(data_request, query_category)
    

    ## 2) If all available, skip, else query needed
    if skip_db_query:
        print("Using Cache")
        database_data = data_request.cache_data
        database_data.columns = [x.lower() for x in database_data.columns]
        data_request.database_data = database_data
        return database_data
    
    ## 2.1) If not all available, query database for missing
    else:
        print("Using DB")
        if query_category == 'single':
            opttick = data_request.query_ticks[0]
            query = f"""SELECT *
            FROM {data_request.db_name}.{data_request.table_name}
            WHERE OPTIONTICK = '{opttick}' AND
            DATETIME >= '{data_request.start_date} 00:00:00' AND 
            DATETIME <= '{data_request.end_date} 00:00:00'
            """
        
        ############# Bulk Query
        elif query_category == 'bulk':
            opttick = data_request.query_ticks
            strikes = data_request.strikes
            opttick_list = [f"{generate_option_tick_new(data_request.symbol, right, data_request.exp, strike)}" for strike, right in strikes]
            data_request.opttick = opttick_list ## save the opttick list for future reference
            str_list = [f"'{x}'" for x in opttick]
            filter_str = f"({', '.join(str_list)})"
            query = f"""SELECT *
            FROM {data_request.db_name}.{data_request.table_name}
            WHERE OPTIONTICK in {filter_str} AND
            DATETIME >= '{data_request.start_date}' AND 
            DATETIME <= '{data_request.end_date}'
            """
        
        ############# Chain Query
        elif query_category == 'chain':
            query = f"""SELECT *
            FROM {data_request.db_name}.{data_request.table_name}
            WHERE TICKER = '{data_request.symbol}' AND
            DATE(BUILD_DATE) = '{data_request.date}' 
            """ 
        
        ## 2.2) Join Cache Data & Missing queried data
        database_data = db.query_database(data_request.db_name, data_request.table_name, query)
        database_data.columns = [x.lower() for x in database_data.columns]
        data_request.query = query
        database_data = pd.concat([database_data, data_request.cache_data])
        data_request.database_data = database_data

        ## 2.3) Update Cache
        
        key = f"{data_request.db_name}.{data_request.table_name}"
        DB_CACHE[key] = pd.concat([
            DB_CACHE[key],
            database_data
        ])
        DB_CACHE[key] = DB_CACHE[key].drop_duplicates(inplace = False)
        return database_data

data = init_query(data_request = chain_request, query_category = 'chain')

Using DB
[get_pymysql_connection] Creating connection for DB: vol_surface, PID: 13289


In [266]:
data.option_tick.unique()

array(['JPM20240920C100', 'JPM20240920C105', 'JPM20240920C110', ...,
       'JPM20270115P280', 'JPM20270115P290', 'JPM20270115P300'],
      dtype=object)

In [219]:
from dbase.database.SQLHelpers import _dispose_all_engines
_dispose_all_engines()

In [208]:
DB_CACHE

<CustomCache 2 entries; sample={'securities_master.temp_options_eod_new':      open  high   low  close  volume  bid_size  closebid  ask_size  closeask  \
0    0.00  0.00  0.00   0.00       0        68      9.10        53      9.50   
1    8.80  8.85  8.80   8.85      41        57      8.70        18      8.95   
2    8.25  8.28  8.25   8.28       9        36      7.80        54      8.10   
3    8.05  8.05  7.85   7.85       5       166      7.30       113      7.90   
4    7.20  7.20  7.20   7.20       3        52      6.90        30      7.20   
..    ...   ...   ...    ...     ...       ...       ...       ...       ...   
718  4.33  4.33  4.33   4.33       1        40      4.30        90      4.50   
719  0.00  0.00  0.00   0.00       0        40      3.85        49      4.10   
720  0.00  0.00  0.00   0.00       0        33      3.85        49      4.10   
721  0.00  0.00  0.00   0.00       0        26      4.30        56      4.50   
722  4.00  4.00  4.00   4.00       2         1

In [2]:


class AttributionDataManager:
    pass



class ScenarioDataManager:
    pass

In [46]:
def flatten_all_dfs(request):
    """
    Flattens all dataframes in the request object.
    """
    for key, value in request.__dict__.items():
        if isinstance(value, pd.DataFrame):
            request.__dict__[key] = value.to_dict(orient="records")
        elif isinstance(value, pd.Series):
            request.__dict__[key] = value.to_dict()
    return request

In [49]:
import json 
def save_failed_request(request):
    """
    Saves the failed request to a JSON file.
    """
    request = flatten_all_dfs(request)
    with open(f'{os.environ["WORK_DIR"]}/module_test/raw_code/DataManagers/failed_request.jsonl', 'a') as f:
        json.dump(request.__dict__, f, default=str)
        f.write('\n')

In [50]:
save_failed_request(request)

## CALCULATE FUNCTIONS

In [3]:
def calc_vol_for_data(
        df,
        price_col,
        col_name,
        model,
        col_kwargs = None
) -> pd.DataFrame:
    """
    Adds a vol column to passed dataframe.

    Parameters:
    df: DataFrame containing following columns: `underlier_price`, `strike`, `expiration`, `datetime`, `rf_rate`, `dividend`, `put/call`
    price_col: Column to back out Implied Vol from
    model: bs or binomial
    col_name: name of added column
    col_kwargs: dictionary with keys as column names in df and values as the corresponding column names in the model
        expected keys: `underlier_price`, `strike`, `expiration`, `datetime`, `rf_rate`, `dividend`, `put/call`

    returns pd.DataFrame
    """
    enforce_allowed_models(model)
    
    if not col_kwargs:
        col_kwargs = {
            'underlier_price': 'underlier_price',
            'strike': 'strike',
            'expiration': 'expiration',
            'datetime': 'datetime',
            'rf_rate': 'rf_rate',
            'dividend': 'dividend',
            'put/call': 'put/call',
        }

    if model == 'bs':
        df[col_name] = df.apply(
        lambda x:IV_handler(S = x[col_kwargs['underlier_price']], 
                            K = x[col_kwargs['strike']], 
                            price = x[price_col], 
                            t = time_distance_helper(x[col_kwargs['expiration']], x[col_kwargs['datetime']]), 
                            r = x[col_kwargs['rf_rate']], 
                            q = x[col_kwargs['dividend']], 
                            flag = x[col_kwargs['put/call']].lower()), axis = 1
    )
        
    elif model == 'binomial':
        df[col_name] = df.apply(
            lambda x: binomial_implied_vol(price = x[price_col], 
                                           S = x[col_kwargs['underlier_price']], 
                                           K = x[col_kwargs['strike']],  
                                           r = x[col_kwargs['rf_rate']], 
                                           exp_date=x[col_kwargs['expiration']], 
                                           option_type=x[col_kwargs['put/call']].lower(), 
                                           pricing_date=x[col_kwargs['datetime']], 
                                           dividend_yield=x[col_kwargs['dividend']]), axis=1
        )
    return df

In [4]:
def calc_vol_for_data_parallel(
        df,
        price_col,
        col_name,
        model,
        col_kwargs = None,
        pool = POOL_ENABLED
) -> pd.DataFrame:
    """
    Adds a vol column to passed dataframe.

    Parameters:
    df: DataFrame containing following columns: `underlier_price`, `strike`, `expiration`, `datetime`, `rf_rate`, `dividend`, `put/call`
    price_col: Column to back out Implied Vol from
    model: bs or binomial
    col_name: name of added column
    col_kwargs: dictionary with keys as column names in df and values as the corresponding column names in the model
        expected keys: `underlier_price`, `strike`, `expiration`, `datetime`, `rf_rate`, `dividend`, `put/call`

    returns pd.DataFrame
    """
    enforce_allowed_models(model)
    
    if not col_kwargs:
        col_kwargs = {
            'underlier_price': 'underlier_price',
            'strike': 'strike',
            'expiration': 'expiration',
            'datetime': 'datetime',
            'rf_rate': 'rf_rate',
            'dividend': 'dividend',
            'put/call': 'put/call',
        }
    temp_df = df.copy()
    temp_df.rename(columns=col_kwargs, inplace=True)
    temp_df['t'] = temp_df.apply(lambda x: time_distance_helper(x[col_kwargs['expiration']], x[col_kwargs['datetime']]), axis=1)
    binomial_column = [price_col, col_kwargs['underlier_price'], 
                       col_kwargs['strike'], col_kwargs['rf_rate'], col_kwargs['expiration'],
                        col_kwargs['put/call'], col_kwargs['datetime'], col_kwargs['dividend'],]
    
    bs_column = [price_col, col_kwargs['underlier_price'], col_kwargs['strike'], 't', col_kwargs['rf_rate'], col_kwargs['dividend'], col_kwargs['put/call']]
    if model == 'bs':
        df[col_name] = parallel_apply(temp_df[bs_column], IV_handler, pool = pool)
        
    elif model == 'binomial':
        df[col_name] = parallel_apply(temp_df[binomial_column], binomial_implied_vol, pool = pool)
    return df

In [5]:
def calc_greeks_for_data(
        df,
        model,
        vol_col,
        greek_name_format,
        col_kwargs = None,
        greek_name = None
) -> pd.DataFrame:
    """
    Adds a greek column to passed dataframe.

    Parameters:
    df: DataFrame containing following columns: `underlier_price`, `strike`, `expiration`, `datetime`, `rf_rate`, `dividend`, `put/call`
    model: bs or binomial
    greek_name: Format of greek name. Eg "Midpoint_BS_{x}" or "Midpoint_binomial_{x}"

    returns pd.DataFrame
    """
    enforce_allowed_models(model) 
    if not col_kwargs:
        col_kwargs = {
            'underlier_price': 'underlier_price',
            'strike': 'strike',
            'expiration': 'expiration',
            'datetime': 'datetime',
            'rf_rate': 'rf_rate',
            'dividend': 'dividend',
            'put/call': 'put/call',
        } 
    
    if not greek_name:
        if model == 'bs':
            greek = df.apply(
                        lambda x:Calculate.greeks(S = x[col_kwargs['underlier_price']], 
                                                K = x[col_kwargs['strike']], 
                                                r = x[col_kwargs['rf_rate']], 
                                                sigma = x[vol_col], 
                                                start = x[col_kwargs['datetime']].strftime('%Y-%m-%d'), 
                                                flag =x[col_kwargs['put/call']].lower(),
                                                exp =   x[col_kwargs['expiration']], 
                                                y = x[col_kwargs['dividend']]), axis = 1, result_type = 'expand'
                    )
        elif model == 'binomial':
            greek = df.apply(
                        lambda x:Calculate.greeks(S = x[col_kwargs['underlier_price']], 
                                                K = x[col_kwargs['strike']], 
                                                r = x[col_kwargs['rf_rate']], 
                                                sigma = x[vol_col], 
                                                start = x[col_kwargs['datetime']].strftime('%Y-%m-%d'), 
                                                flag =x[col_kwargs['put/call']].lower(), 
                                                exp =   x[col_kwargs['expiration']], 
                                                y = x[col_kwargs['dividend']],
                                                model = model), axis = 1, result_type = 'expand'
                    )
        greek.columns = [greek_name_format.format(x=x) for x in greek.columns]
        df[greek.columns] = greek
        return df
    else:
        calc_func = getattr(Calculate, greek_name.lower())
        greek = df.apply(
                    lambda x:calc_func(S = x[col_kwargs['underlier_price']], 
                                       K = x[col_kwargs['strike']], 
                                       r = x[col_kwargs['rf_rate']], 
                                       sigma = x[vol_col], 
                                       start = x[col_kwargs['datetime']].strftime('%Y-%m-%d'), 
                                       flag =x[col_kwargs['put/call']].lower(), 
                                       exp =   x[col_kwargs['expiration']], 
                                       y = x[col_kwargs['dividend']]), axis = 1)
        
        df[greek_name_format.format(x=greek_name)] = greek
        return df





In [6]:
def calc_greeks_for_data_parallel(
        df,
        model,
        vol_col,
        greek_name_format,
        col_kwargs = None,
        greek_name = None,
        pool = POOL_ENABLED
) -> pd.DataFrame:
    """
    Adds a greek column to passed dataframe.

    Parameters:
    df: DataFrame containing following columns: `underlier_price`, `strike`, `expiration`, `datetime`, `rf_rate`, `dividend`, `put/call`
    model: bs or binomial
    greek_name: Format of greek name. Eg "Midpoint_BS_{x}" or "Midpoint_binomial_{x}"

    returns pd.DataFrame
    """
    enforce_allowed_models(model) 
    if not col_kwargs:
        col_kwargs = {
            'underlier_price': 'underlier_price',
            'strike': 'strike',
            'expiration': 'expiration',
            'datetime': 'datetime',
            'rf_rate': 'rf_rate',
            'dividend': 'dividend',
            'put/call': 'put/call',
        } 

    temp_df = df.copy()
    temp_df.rename(columns=col_kwargs, inplace=True)
    temp_df['t'] = temp_df.apply(lambda x: time_distance_helper(x[col_kwargs['expiration']], x[col_kwargs['datetime']]), axis=1)
    temp_df['asset'] = None
    temp_df['model'] = model
    greeks_colums_use = ['asset',col_kwargs['underlier_price'], col_kwargs['strike'], 
                         col_kwargs['rf_rate'], 'midpoint_binomial_iv',
                         col_kwargs[ 'datetime'], col_kwargs['put/call'], col_kwargs['expiration'], col_kwargs['dividend'], 'model']
                         
    if not greek_name:
        greek = parallel_apply(temp_df[greeks_colums_use], Calculate.greeks, pool = pool)
        greek = pd.DataFrame(greek)
        greek.columns = [greek_name_format.format(x=x) for x in greek.columns]
        greek.index = temp_df.index
        df[greek.columns] = greek
        return df
    else:
        calc_func = getattr(Calculate, greek_name.lower())
        greek = parallel_apply(temp_df[greeks_colums_use], calc_func, pool = pool)
        df[greek_name_format.format(x=greek_name)] = greek
        return df


In [7]:
def calc_dollar_delta_from_data(
        df,
        delta_col,
        col_name   
) -> pd.DataFrame:
    """
    Adds a Dollar Delta Column to passed dataframe

    parameters:
    df: DataFrame containing following columns: `underlier_price`, `strike`, `expiration`, `datetime`, `rf_rate`, `dividend`, `put/call`
    delta_col: Delta Column to use in multiplication
    col_name: Format for added columns. Eg "midpoint_dollar_delta"
    """

    df[col_name] = df['underlier_price'] * df[delta_col]
    return df

In [8]:
def resolve_missing_vols_in_data(
        df,
        vol_resolve_list,
        model_map_list,
        price_map_list,
        agg,
):
    """
    Resolves missing vols in passed dataframe

    parameters:
    df: DataFrame containing following columns: `underlier_price`, `strike`, `expiration`, `datetime`, `rf_rate`, `dividend`, `put/call`
    vol_resolve_list: List of columns to resolve missing vols in
    model_map_list: List of models to use for resolving missing vols. Maps according to vol_resolve_list index
    price_map_list: List of columns to use for pricing. Maps according to vol_resolve_list index
    """
    for col, model, price_col in zip(vol_resolve_list, model_map_list, price_map_list):
        zero_vol = df[col] == 0
        resolved_vols = df[zero_vol].apply(
                lambda x:resolve_missing_vol(
                    underlier = x['underlier'],
                    expiration = x['expiration'],
                    strike = x['strike'],
                    put_call = x['put/call'],
                    datetime = x['datetime'],
                    S = x['underlier_price'],
                    r = x['rf_rate'],
                    q = x['dividend'],
                    pricing_model = model,
                    agg = agg,
            ), axis = 1)
        df.loc[zero_vol, col] = resolved_vols
        new_pv = df[zero_vol].apply(
            lambda x: optionPV_helper(
                spot_price = x['underlier_price'],
                strike_price = x['strike'],
                exp_date = x['expiration'],
                risk_free_rate = x['rf_rate'],
                dividend_yield = x['dividend'],
                volatility = x[col],
                putcall = x['put/call'],
                settlement_date_str= x['datetime'],
                model = model,
            ), axis = 1
        )
        df.loc[zero_vol, price_col] = new_pv

        ## After resolving Vols, we will have to recalculate the specific price
        
    return df

In [9]:
__all__ = [
    'DataManagerBase',
    'OptionDataManager',
    'SpotDataManager',
    'VolDataManager',
    'GreeksDataManager',
    'AttributionDataManager',
]

## MANAGERLAZYLOADER

In [11]:
class _ManagerLazyLoader:
    def __init__(self, symbol):
        self.symbol = symbol
        self.Stock = Stock(self.symbol, run_chain = False)
        self._eod = {}
        self._intra = {}


    @property
    def eod(self):
        """
        Returns the end of day data
        """
        class EODData(dict):
            def __init__(inner, parent): ## inner is the instance of the class, parent is the instance of the parent class
                inner.parent = parent
                super().__init__()

            def __getitem__(inner, key): ## Custom getter for EOD Dict. To initialize the data, if not already done
                if key not in inner.parent._eod:
                    if key not in ['s0_close', 's0_chain', 'r', 'y', 'r_name']:
                        raise KeyError(f"{key} not in eod data, expected one of: ['s0_close', 's0_chain', 'r', 'y', 'r_name]")
                    inner.parent._eod[key] = inner.parent._lazy_load(key, intra_flag = False)
                return inner.parent._eod[key]
            
            def __contains__(innner, key):
                return key in inner.parent._eod
            
            def __repr__(inner):
                return inner.parent._eod.__repr__()
            
            def __len__(inner):
                return len(inner.parent._eod)
            
            def keys(inner):
                return inner.parent._eod.keys()
        return EODData(self)
    
    @property
    def intra(self):
        """
        Returns the end of day data
        """
        class IntraData(dict):
            def __init__(inner, parent):
                inner.parent = parent
                super().__init__()

            def __getitem__(inner, key): ## Custom getter for EOD Dict. To initialize the data, if not already done
                if key not in inner.parent._intra:
                    if key not in ['s0_close', 's0_chain', 'r', 'y', 'r_name']:
                        raise KeyError(f"{key} not in intra data, expected one of: ['s0_close', 's0_chain', 'r', 'y', 'r_name']")
                    inner.parent._intra[key] = inner.parent._lazy_load(key, ts_timewidth = '5', ts_timeframe = 'minute', intra_flag = True)
                return inner.parent._intra[key]
            
            def __contains__(innner, key):
                return key in inner.parent._intra
            
            def __repr__(inner):
                return inner.parent._intra.__repr__()
            
            def __len__(inner):
                return len(inner.parent._intra)
            
            def keys(inner):
                return inner.parent._intra.keys()
        return IntraData(self)


    def _lazy_load(self, load_name, **kwargs):
        ## Utilizing the lazy load function to load data on demand, and speed up initialization
        if load_name == 's0_close':

            ## Will use Kwargs to move between intra and EOD.
            kwargs.pop('intra_flag')
            return_item =  (self.Stock.spot(ts = True,
                                          ts_start = pd.to_datetime(self.exp) - relativedelta(years=5),
                                          ts_end =pd.to_datetime(self.exp) + relativedelta(years=5),
                                          **kwargs))
            return return_item
        
        elif load_name == 's0_chain':
            kwargs.pop('intra_flag')
            return_item =  (self.Stock.spot(ts = True,
                                            ts_start = pd.to_datetime(self.exp) - relativedelta(years=5),
                                            ts_end =pd.to_datetime(self.exp) + relativedelta(years=5),
                                            spot_type='chain_price',
                                            **kwargs))
            return return_item
            
        elif load_name == 'r':
            intra_flag = kwargs.get('intra_flag', False)
            r = (get_risk_free_rate_helper()['annualized'])
            if intra_flag:
                return resample(r, PRICING_CONFIG['INTRADAY_AGG'], {'risk_free_rate':'ffill'})
            else:
                return r
    
        elif load_name == 'r_name':
            intra_flag = kwargs.get('intra_flag', False)
            r = (get_risk_free_rate_helper()['name'])

            if intra_flag:
                return resample(r, PRICING_CONFIG['INTRADAY_AGG'], {'risk_free_rate':'ffill'})
            else:
                return r

        elif load_name == 'y':
            ## Get the dividend yield
            intra_flag = kwargs.get('intra_flag', False)
            y = (self.Stock.div_yield_history(start = pd.to_datetime(self.exp) - relativedelta(years=2)))

            if intra_flag:
                return resample(y, PRICING_CONFIG['INTRADAY_AGG'], {'dividend_yield':'ffill'})
            else:
                return y
        
    

## VOL SPOT GREEK MANAGER

In [12]:

class SpotDataManager:
    def __init__(self, symbol:str):
        self.symbol = symbol

    def query_thetadata(self,
                        start: str | datetime,
                        end: str | datetime,
                        strike: float = None,
                        exp: str | datetime = None,
                        right: str = None,
                        bulk: bool = False,
                        **kwargs) -> pd.DataFrame:
        """
        Query the spot data & Open Interest from ThetaData API.
        """
        data_request = kwargs.get('data_request')
        print_url = kwargs.get('print_url', False)
        agg = data_request.agg
        if agg == 'eod':
            if not bulk:
                data = retrieve_eod_ohlc(symbol=self.symbol, end_date=end, exp=exp, right=right, start_date=start, strike=strike, print_url=print_url)
                data = data[~data.index.duplicated(keep='first')]
                open_interest = retrieve_openInterest(symbol=self.symbol, end_date=end, exp=exp, right=right, start_date=start, strike=strike, print_url=print_url).set_index('Datetime')
                data['Open_interest'] = open_interest['Open_interest']
                data.index = default_timestamp(data.index)
                return data

            else:
                bulk = retrieve_bulk_eod(
                    symbol = self.symbol,
                    exp = exp,
                    start_date = start,
                    end_date = end,
                )

                ## Add Option Tick
                bulk_eod = bulk.reset_index()
                tick_col = ['Root', 'Right', 'Expiration', 'Strike']
                bulk_eod['OptionTick'] = parallel_apply(bulk_eod[tick_col], generate_option_tick_new, pool = POOL_ENABLED)
                if data_request.opttick is not None:
                    bulk_eod = bulk_eod[bulk_eod['OptionTick'].isin(data_request.opttick)]


                ## Query Bulk Open Interest
                bulk_oi = retrieve_bulk_open_interest(
                    symbol = self.symbol,
                    exp = exp,
                    start_date = start,
                    end_date = end,
                )
                ## Add Option Tick
                bulk_oi['OptionTick'] = parallel_apply(bulk_oi[tick_col], generate_option_tick_new, pool = POOL_ENABLED)
                if data_request.opttick is not None:
                    bulk_oi = bulk_oi[bulk_oi['OptionTick'].isin(data_request.opttick)]
                ## Add EOD Timestamp
                bulk_oi['Datetime'] = add_eod_timestamp(pd.DatetimeIndex(bulk_oi['Datetime']))
                data = bulk_eod.merge(bulk_oi[['Datetime','OptionTick', 'Open_interest']], on = ['Datetime', 'OptionTick'], how = 'left')
                data = data.rename(columns = {'Root': 'ticker', 'Strike':'k', 'Expiration': 'exp_date'})
                data.set_index('Datetime', inplace = True)
                data.index = default_timestamp(pd.DatetimeIndex(data.index))
                return data
            
        elif agg == 'intra':
            if not bulk:
                data = retrieve_ohlc(symbol=self.symbol, end_date=end, exp=exp, right=right, start_date=pd.to_datetime(start) - BDay(1), strike=strike, print_url=print_url)
                ## For open Interest we will query from Start - 1BDay to End
                open_interest = retrieve_openInterest(symbol=self.symbol, end_date=end, exp=exp, right=right, 
                                                      start_date=pd.to_datetime(start) - BDay(1), strike=strike, 
                                                      print_url=print_url).set_index('Datetime')['Open_interest']
                
                ## PS: Quering for Open Interest uses Start - 1BDAY to END
                ## This is because open interest returns EOD. Resampling to intraday moves previous day data to current day
                ## Therefore, first date will be NaN because it will be the previous day data, which is not included in the query results
                open_interest = resample(open_interest, PRICING_CONFIG['INTRADAY_AGG'] )
                data['Open_interest']=open_interest
                
                return data#.dropna()
                # return open_interest
            else:
                raise NotImplementedError("Bulk data not implemented for intra data")
            

class VolDataManager:
    def __init__(self, symbol:str):
        self.symbol = symbol

    def calculate_iv(self, **kwargs):
        """
        Calculate the implied volatility using the model.
        """
        data_request = kwargs['data_request']
        model = data_request.model
        raw_data = data_request.raw_spot_data
        raw_data.columns = [x.lower() for x in raw_data.columns]
        raw_data['datetime'] = raw_data.index
        return_cols = []
        for col, name in data_request.iv_cols.items():
            calc_vol_for_data_parallel(raw_data, col, name, model, col_kwargs = data_request.col_kwargs, pool = False)

        raw_data.drop(columns=['datetime'], inplace=True)


class GreeksDataManager:
    def __init__(self, symbol:str):
        self.symbol = symbol

    def calculate_greeks(self, type_, **kwargs):
       
        data_request = kwargs['data_request']
        model = data_request.model
        raw_data = data_request.raw_spot_data
        raw_data.columns = [x.lower() for x in raw_data.columns]
        raw_data['datetime'] = raw_data.index
        if type_ in ['greek', 'greeks']:
            ## Greeks
            for col, format_name in data_request.greek_cols.items():
                calc_greeks_for_data_parallel(raw_data, model, col, format_name, col_kwargs = data_request.col_kwargs, pool = False)
        else:
            ## Individual Greeks
            for col, format_name in data_request.greek_cols.items():
                calc_greeks_for_data_parallel(raw_data, model, col, format_name, col_kwargs = data_request.col_kwargs, greek_name=type_, pool = False)
        
        raw_data.drop(columns=['datetime'], inplace=True)


## CHAIN MANAGER

In [45]:
class ChainDataRequest:
    def __init__(self, symbol, date, table_name, db_name):
        self.symbol = symbol
        self.date = date
        self.table_name = table_name
        self.db_name = db_name
        


## Writing this as a separate class to handle the chain data
## I don't want it to depend on OptionDataManager
## Because it isn't tethered to a specific option.
class ChainDataManager(_ManagerLazyLoader):
    """
    Class to manage the chain data for a given symbol.
    It inherits from the _ManagerLazyLoader class to load data on demand.
    It uses the ChainDataRequest class to handle the data requests.
    It uses the DatabaseAdapter class to handle the database operations.
    """
    def __init__(self, symbol):
        """
        Initialize the ChainDataManager with the symbol.
        """
        super().__init__(symbol)
        self.symbol = symbol
        self.requests = {}
        self.current_request = ''
        self.db = DatabaseAdapter()


    def get_at_time(self, date:str, organize:bool = False) -> pd.DataFrame:
        database, table = TABLES['eod']['chain'].split('.')
        self.current_request = datetime.now().strftime("%Y%m%d %H:%M:%S")
        data_request = ChainDataRequest(
            symbol=self.symbol,
            date = date,
            table_name = table,
            db_name = database )
        self.requests[self.current_request] = data_request
        init_query(data_request=data_request, db=self.db, query_category='chain')
        self.__pre_process(data_request=data_request)
        is_empty = data_request.is_empty
        if is_empty:
            print(f"Data for {self.symbol} on {date} is empty")
            self.__post_process(data_request=data_request)
            save_thread = Thread(target=self.save_chain_data, args=(data_request,), daemon=True, name = "save_chain_data")
            save_thread.start()
            data_request.save_thread = save_thread
            CENTRAL_SAVE_THREAD[self.current_request] = save_thread

        else:
            data_request.post_processed_data = data_request.database_data

        if organize:
                data = data_request.post_processed_data.copy()
                data.columns = data.columns.str.capitalize() 
                data.rename(columns = {'Dte': 'DTE', 'Price': 'Midpoint'}, inplace=True)
                chain = data.pivot_table(
                                    index = ['Expiration', 'DTE', 'Strike'],
                                    columns = ['Right'],
                                    values = ['Midpoint']
                                )
                data_request.organized_data = chain
        else:
            data_request.organized_data = data_request.post_processed_data
        
        return data_request
                
            

    def __pre_process(self, **kwargs):
        """
        Preprocess the data for the request
        """
        data_request = kwargs.get('data_request')
        database_data = data_request.database_data
        database_data.columns = [x.lower() for x in database_data.columns]
        if database_data.empty:
            data_request.is_empty = True
        else:
            data_request.is_empty = False
        
        return database_data

    def __post_process(self, **kwargs):
        """
        Postprocess the data for the request
        """
        logger.warning(f"ChainDataManger will not be returning Volatility data due to performance.")
        data_request = kwargs.get('data_request')
        date = data_request.date
        chain = retrieve_chain_bulk(self.symbol, 0, date, date, PRICING_CONFIG['MARKET_CLOSE_TIME'])
        chain.index.name = 'build_date'
        self.exp = chain['Expiration'].unique()[0] ## Setting Expiration Date as an instance variable so LazyLoaderManager can use it
        chain_v2 = chain.rename(columns = {'Root':'ticker', 'Midpoint': 'price'}).drop(columns = ['Date']).reset_index()
        chain_v2.columns = [x.lower() for x in chain_v2.columns]
        chain_v2['dte'] = (chain_v2['expiration'] - chain_v2['build_date']).dt.days
        chain_v2['spot'] = self.eod['s0_chain']['close'][date]
        chain_v2['r'] = self.eod['r'][date]
        chain_v2['q'] = self.eod['y'][date]
        chain_v2['option_tick'] = chain_v2.apply(lambda x: generate_option_tick_new(x['ticker'], x['right'], x['expiration'].strftime('%Y-%m-%d'), x['strike']), axis=1)
        chain_v2['moneyness'] = chain_v2.apply(lambda x: x['spot'] / x['strike'], axis=1)
        data_request.post_processed_data = chain_v2

    def save_chain_data(self, data_request, **kwargs):
        """
        Save the chain data to the database
        """
        col_kwargs = {
            'underlier_price': 'spot',
            'strike': 'strike',
            'expiration': 'expiration',
            'datetime': 'build_date',
            'rf_rate': 'r',
            'dividend': 'q',
            'put/call': 'right'
        }
        chain_data = data_request.post_processed_data
        calc_vol_for_data(chain_data, 'price', 'bs_vol', 'bs', col_kwargs=col_kwargs)
        binomial_col = ['price', 'spot', 'strike', 'r', 'expiration', 'right', 'build_date', 'q']
        chain_data['binomial_vol'] = parallel_apply(chain_data[binomial_col], binomial_implied_vol, timeout=10, pool=POOL_ENABLED)

        self.db.save_to_database(chain_data, data_request.db_name, data_request.table_name,)


chain_manager_ = ChainDataManager('AAPL')
request = chain_manager_.get_at_time('2022-01-21', organize=False)


Using Function to query the database


In [94]:
del chain_manager

## MISC FUNC

In [14]:
def handle_extra_cols(extra_cols, type_, model):
        return_cols = []
        if extra_cols:
            assert all([x in ['ask', 'bid', 'open'] for x in extra_cols]), f"Expected extra_cols to be one of: ['ask', 'bid', 'open'] received {extra_cols}"
        
        if type_ == 'spot':
              for col in extra_cols:
                if col == 'ask':
                     return_cols.append('closeask')
                elif col == 'bid':
                    return_cols.append('closebid')
                elif col == 'open':
                     return_cols.append('open')

        elif type_ == 'vol':
            for col in extra_cols:
                if col == 'ask':
                     return_cols.append(f'ask_{model}_iv')
                elif col == 'bid':
                    return_cols.append(f'bid_{model}_iv')
                elif col == 'open':
                     raise ValueError("Open not implemented for vol data")
                
        elif type_ in ['greeks', 'greek']:
            if model == 'bs':
                  raise ValueError("Extra Cols not implemented for BS Greeks")
            elif model == 'binomial':
                 for col in extra_cols:
                    if col == 'ask':
                        return_cols.extend([f'ask_{model}_{x}' for x in PRICING_CONFIG['AVAILABLE_GREEKS']])
                    elif col == 'bid':
                        return_cols.extend([f'bid_{model}_{x}' for x in PRICING_CONFIG['AVAILABLE_GREEKS']])
                    elif col == 'open':
                        raise ValueError("Open not implemented for greek data")
                        
        elif type_ in PRICING_CONFIG['AVAILABLE_GREEKS']:
            if model == 'bs':
                  raise ValueError("Extra Cols not implemented for BS Greeks")
            for col in extra_cols:
                if col == 'ask':
                    return_cols.append(f'ask_{model}_{type_}')
                elif col == 'bid':
                    return_cols.append(f'bid_{model}_{type_}')
                elif col == 'open':
                     raise ValueError("Open not implemented for vol data")
             
        return return_cols

In [15]:
def build_name_format(type_, model, extra_cols, default_fill):
    """
    Build the name format for the columns
    """
    name_format = {}

    if type_ == 'vol':
        if model == 'bs':
            name_format['close'] = 'bs_iv'
            name_format[f"{default_fill}"] = f"{default_fill}_bs_iv"

            ## Handle extra columns
            for col in extra_cols:
                if col.lower() in ['open']:
                    continue
                name_format[f"close{col}"] = handle_extra_cols([col], type_, model)[0]


        elif model == 'binomial':
            name_format['close'] = 'binomial_iv'
            name_format[f"{default_fill}"] = f"{default_fill}_binomial_iv"
            for col in extra_cols:
                if col.lower() in ['open']:
                    continue
                name_format[f"close{col}"] = handle_extra_cols([col], type_, model)[0]
    
    elif type_ in ['greek', 'greeks']:
        if model == 'bs':
            name_format['bs_iv'] = '{x}'
            name_format[f"{default_fill}_bs_iv"] = f'{default_fill}_'+'{x}'
            if extra_cols:
                pass ## Figure out how to handle extra cols
            
        elif model == 'binomial':
            name_format['binomial_iv'] = 'binomial_{x}'
            name_format[f"{default_fill}_binomial_iv"] = f'{default_fill}_binomial_'+'{x}'
            for col in extra_cols:
                name_format[f"{col}_binomial_iv"] = f"{col}_{model}_" +"{x}"
    
    elif type_ in PRICING_CONFIG['AVAILABLE_GREEKS']:
        if model == 'bs':
            name_format['bs_iv'] = '{x}'
            name_format[f"{default_fill}_bs_iv"] = f'{default_fill}_'+'{x}'
        elif model == 'binomial':
            name_format['binomial_iv'] = 'binomial_{x}'
            name_format[f"{default_fill}_binomial_iv"] = f'{default_fill}_binomial_'+'{x}'
            for col in extra_cols:
                name_format[f"{col}_binomial_iv"] = f"{col}_{model}_" +"{x}"


    return name_format

In [16]:
build_name_format('delta',  'binomial', ['ask', 'bid'], 'midpoint')

{'binomial_iv': 'binomial_{x}',
 'midpoint_binomial_iv': 'midpoint_binomial_{x}',
 'ask_binomial_iv': 'ask_binomial_{x}',
 'bid_binomial_iv': 'bid_binomial_{x}'}

## OPTION DATA MANAGER

In [17]:
class OptionQueryRequestParameter:
    def __init__(self, table_name, db_name, start_date=None, end_date=None, ticker=None, exp=None, strike=None, right = None):
        self.db_name = db_name
        self.table_name = table_name
        self.start_date = start_date
        self.end_date = end_date
        self.exp = exp
        self.strike  = strike
        self.right = right
        self.symbol = ticker
        self.opttick= None
        self.query = None
        self.y = None
        self.vol = None
        self.spot = None
        self.interval = None
        self.type_ = None

In [18]:
def enforce_interval(ivl_str: str):
        ## Enforce the interval
    try:
        ## We want to throw an error if the interval is not in the available intervals + if we get query for minute data 'm'
        PRICING_CONFIG['AVAILABLE_INTERVALS'].remove('m') ## Remove minute data from available intervals 
    except:
        pass
    

    if ivl_str.lower() not in PRICING_CONFIG["AVAILABLE_INTERVALS"] and ivl_str != 'M': ## Want to avoid minute data
        raise ValueError(f"Expected interval to be one of: {PRICING_CONFIG['AVAILABLE_INTERVALS']}")
    
    if ivl_str == 'm': ## Minute data not available
        raise AttributeError("Minute data currently unavailable, please go higher")
    
    return

def enforce_inputs(type_:str, model:str) -> None:
        ## Assert inputs
    if type_ not in ['spot', 'vol', 'vega', 'vanna', 'volga', 'delta', 'gamma', 'theta', 'rho', 'greeks', 'greek', 'attribution', 'scenario']:
        raise ValueError("Expected type_ to be one of: ['spot', 'vol', 'vega', 'vanna', 'volga', 'delta', 'gamma', 'theta', 'rho', 'greeks', 'greek', 'attribution', 'scenario']")
    if model not in ['bs', 'binomial', 'mc']: ## Only Black Scholes, binomial tree, monte carlo
        raise ValueError(f"Expected model to be one of: {PRICING_CONFIG['AVAILABLE_PRICING_MODELS']}")
    return

def determine_table_agg(ivl_str: str, type_: str, greek_names: list) -> tuple:
    ## Determine aggregation
    if ivl_str in ['h', 'm']:
        agg = 'intra'
    else:
        agg = 'eod'
    
    ## Table to query, picking based on interval & type
    if type_ in greek_names:
        database, table = TABLES[agg]['greeks'].split('.')
    else:
        database, table = TABLES[agg][type_].split('.')
    
    return agg, database, table


def determine_requested_columns(default_fill:str, type_:str, model:str, greek_names:list) -> list:
    if type_ == 'spot':
        requested_col = ['datetime', 'open', 'high', 'low', 'close', default_fill.lower(),'volume', 'openinterest']

    elif type_ == 'vol':
        requested_col = ['datetime', f"{model}_iv".lower(), f"{default_fill.lower()}_{model}_iv".lower()]

    elif type_ in greek_names:
        ## If Statement logic to format a the list of greek names to be used
        if type_ not in ['greek', 'greeks']:
            if model == 'bs':
                requested_col = ['datetime'] + [f"{default_fill}_{type_}".lower()] + [f"{type_}".lower()]
            else:
                requested_col = ['datetime'] + [f"{model}_{type_}".lower()] + [f"{default_fill.lower()}_{model}_{type_}".lower()]
        else:
            if model == 'bs':
                requested_col = ['datetime'] + [f"{x}".lower() for x in greek_names if x not in ['greek', 'greeks']] + [f"{default_fill.lower()}_{x}".lower() for x in greek_names if x not in ['greek', 'greeks']]
            else:
                requested_col = ['datetime'] + [f"{model}_{x}".lower() for x in greek_names if x not in ['greek', 'greeks']] + [f"{default_fill.lower()}_{model}_{x}".lower() for x in greek_names if x not in ['greek', 'greeks']]

    elif type_ == 'attribution':
        raise NotImplementedError("Attribution data not implemented yet")
    
    elif type_ == 'scenario':
        raise NotImplementedError("Scenario data not implemented yet")
    
    elif type_ == 'chain':
        raise IncorrectExecutionError("Chain Data does not return a timeseries, returns at_time")
    else:
        raise KeyError(f"Type {type_} not in requested columns")
    return requested_col

        
def format_raw_spot_data( **kwargs):
    """
    Adds necessary formatting. To avoid overpopulating the __handle_incomplete_data method
    """
    data_request = kwargs.get('data_request')
    raw_spot_data = data_request.raw_spot_data
    if raw_spot_data.empty:
        print("Format raw found this empty")
        data_request.raw_spot_data = pd.DataFrame()

    else:
        raw_spot_data.reset_index(inplace = True)
        raw_spot_data.columns = [x.lower() for x in raw_spot_data.columns]
        raw_spot_data = raw_spot_data[raw_spot_data.datetime.isin(data_request.missing_dates)]
        if 'index' in raw_spot_data.columns:
            raw_spot_data.drop(columns=['index'], inplace=True)
        data_request.raw_spot_data = raw_spot_data


def post_process( **kwargs):
    """
    Post process the data after all the calculations
    """

    data_request = kwargs.get('data_request')
    bulk = kwargs.get('bulk', False)
    is_complete = data_request.pre_process['is_complete']
    is_empty = data_request.pre_process['is_empty']
    raw_spot_data = data_request.raw_spot_data.copy()

    if not is_empty and is_complete:
        ## If not empty and data complete, no need formatting. Just return from db
        final_data = data_request.pre_processed_data.copy()
        data_request.post_processed_data = final_data
        return
    
    ## Start by renaming the columns to match the database.
    rename_columns = {'open_interest': 'openinterest'}
    try:
        raw_spot_data.rename(columns=rename_columns, inplace=True)
    except KeyError as e:
        pass
    
    ## Filter the columns to only the requested columns
    raw_spot_data = raw_spot_data[data_request.requested_col]

    ## Capitalize the columns & set the index to datetime
    raw_spot_data.columns = [x.capitalize() for x in raw_spot_data.columns]
    if bulk:
        raw_spot_data.set_index(['Optiontick','Datetime'], inplace=True)
    else:
        raw_spot_data.set_index('Datetime', inplace=True)
    raw_spot_data = raw_spot_data[~raw_spot_data.index.duplicated(keep='first')]
    
    if is_empty:
        ## If the data is empty, the final data is the raw data
        final_data = raw_spot_data
    elif not is_complete:
        ## Else we will have to merge the data
        final_data = pd.concat([data_request.pre_processed_data, raw_spot_data], axis=0)
    
    
    final_data = final_data[~final_data.index.duplicated(keep='first')]
    final_data.columns = [x.capitalize() for x in final_data.columns]
    final_data.sort_index(inplace=True)

    if bulk:
        ## For final data, we will filter for the Option Tick we need
        final_data = final_data[final_data.index.get_level_values('Optiontick').isin(data_request.opttick)]

    data_request.post_processed_data = final_data
    return data_request.post_processed_data


def format_final_data(**kwargs):
    """
    Format the final data to match the database
    """
    data_request = kwargs.get('data_request')
    bulk = kwargs.get('bulk', False)
    print("Resampling intra data")
    ## Resample the data to the requested interval
    resampled = resample( data_request.post_processed_data, 
                                                    data_request.interval, 
                                                    {col: 'ffill' for col in data_request.post_processed_data.columns})
    if bulk:
        resampled.index = resampled.index.swaplevel()
    data_request.post_processed_data = resampled

def init_query(**kwargs):

    """
    Initialize the query for the data request and save the data to the request
    """
    print("Using Function to query the database")
    data_request = kwargs.get('data_request')
    db = kwargs.get('db', DatabaseAdapter())
    try:
        query_category = kwargs['query_category']
    except KeyError:
        raise KeyError("Query category not specified, expected one of: ['single', 'bulk', 'chain']")
    
    if query_category == 'single':
        query = f"""SELECT *
        FROM {data_request.db_name}.{data_request.table_name}
        WHERE OPTIONTICK = '{data_request.opttick}' AND
        DATETIME >= '{data_request.start_date}' AND 
        DATETIME <= '{data_request.end_date}'
        """
        database_data = db.query_database(data_request.db_name, data_request.table_name, query)
        database_data.columns = [x.lower() for x in database_data.columns]
        data_request.query = query
        data_request.database_data = database_data
        return database_data
    
    ############# Bulk Query
    elif query_category == 'bulk':
        strikes = data_request.strikes
        opttick_list = [f"{generate_option_tick_new(data_request.symbol, right, data_request.exp, strike)}" for strike, right in strikes]
        data_request.opttick = opttick_list ## save the opttick list for future reference
        str_list = [f"'{x}'" for x in opttick_list]
        filter_str = f"({', '.join(str_list)})"
        query = f"""SELECT *
        FROM {data_request.db_name}.{data_request.table_name}
        WHERE OPTIONTICK in {filter_str} AND
        DATETIME >= '{data_request.start_date}' AND 
        DATETIME <= '{data_request.end_date}'
        """
        database_data = db.query_database(data_request.db_name, data_request.table_name, query)
        database_data.columns = [x.lower() for x in database_data.columns]
        data_request.query = query
        data_request.database_data = database_data
        return database_data
    
    ############# Chain Query
    elif query_category == 'chain':
        query = f"""SELECT *
        FROM {data_request.db_name}.{data_request.table_name}
        WHERE TICKER = '{data_request.symbol}' AND
        BUILD_DATE = '{data_request.date} 16:00:00'
        """
        
        database_data = db.query_database(data_request.db_name, data_request.table_name, query)
        database_data.columns = [x.lower() for x in database_data.columns]
        data_request.query = query
        data_request.database_data = database_data
        return database_data
    

def add_inputs_to_raw(self, **kwargs):
    """
    Adds Inputs to raw_spot_data for Vol & other necessary uses
    """
    data_request = kwargs.get('data_request')
    bulk = kwargs.get('bulk', False)
    if not bulk:
        if not data_request.raw_spot_data.empty:
            data_request.raw_spot_data['s0'] = data_request.input_params['s0_chain']['close']
            data_request.raw_spot_data['y'] = data_request.input_params['y']
            data_request.raw_spot_data['r'] = data_request.input_params['r']
            data_request.raw_spot_data['K'] = self.strike
            data_request.raw_spot_data['exp_date'] = self.exp
            data_request.raw_spot_data['right'] = self.right
    else:
        if not data_request.raw_spot_data.empty:
            data_request.raw_spot_data['s0'] = data_request.input_params['s0_chain']['close']
            data_request.raw_spot_data['y'] = data_request.input_params['y']
            data_request.raw_spot_data['r'] = data_request.input_params['r']


In [19]:
class OptionDataManager(_ManagerLazyLoader):
    @log_time(time_logger)
    def __init__(self,
                symbol: str = None,
                exp: str | datetime = None,
                right: str = None,
                strike: float = None,
                default_fill: str = 'midpoint',
                opttick: str = None,
                **kwargs) -> None:
        """
        Returns an object for querying data

        Params:
        symbol: Underlier symbol
        exp: expiration
        right: Put(P) or Call (C)
        strike: Option Strike
        default_fill: How to fill zero values for close. 'midpoint' or 'weighted_midpoint'
        opttick: Option ticker, if provided, will ignore symbol, exp, right, strike and be initialized with the string
        """

        super().__init__(symbol)
        if opttick is not None:
            assert isinstance(opttick, str), f"opttick has to be type str, recieved {type(opttick)}"
            option_meta = parse_option_tick(opttick)
            self.symbol = option_meta['ticker']
            self.exp = option_meta['exp_date']
            self.right = option_meta['put_call']
            self.strike = option_meta['strike']
            self.opttick = opttick

        else:
            assert isinstance(strike, float), f"Strike has to be type float, recieved {type(strike)}"
            if default_fill not in ['midpoint', 'weighted_midpoint', None]:
                raise ValueError("Expected default_fill to be one of: 'midpoint', 'weighted_midpoint', None ")
            
            assert all([symbol, exp, right, strike]), "symbol, exp, right, strike are required"
            self.exp = exp
            self.symbol = symbol
            self.right = right.upper()
            self.strike = strike
            self.opttick = generate_option_tick_new(symbol, right, exp, strike)

        self.default_fill = default_fill
        self.db = DatabaseAdapter()
        self.data_request = {}
        self.save_thread = {}
        self.current_request =None
        self.spot_manager = SpotDataManager(self.symbol)
        self.vol_manager = VolDataManager(self.symbol)
        self.greek_manager = GreeksDataManager(self.symbol)
        self.chain_manager = ChainDataManager(self.symbol)
        self.greek_names = PRICING_CONFIG["AVAILABLE_GREEKS"] + ['greek', 'greeks']
        self.print_info = kwargs.get('print_info', False)

        ## Prefer to use dicts to avoid having too many attributes

    def get_timeseries(self, 
                       start: str | datetime, 
                       end: str | datetime,
                       interval: str = '1d',
                       type_: str = 'spot',
                       model: str = 'bs',
                       extra_cols: list = []) -> pd.DataFrame:
        """
        Query the timeseries data from ThetaData API or SQL Database.

        Params:
        start: Start date for the query
        end: End date for the query
        interval: Interval for the query. Options are: h, d, w, M, q, y
        type_: Type of data to query. Options are: spot, vol, greeks, greek, attribution, scenario
        model: Model to use for the query. Options are: bs, binomial
        extra_cols: Extra columns to include in the query. Options are: ask, bid, open
        """
        
        
        ## Organize inputs
        start = pd.to_datetime(start)
        end = pd.to_datetime(end)
        ivl_str, ivl_int = extract_numeric_value(interval)
        greek_names = self.greek_names
        self.current_request = datetime.now().strftime("%Y%m%d %H:%M:%S")
        _extra_cols = handle_extra_cols(extra_cols, type_, model)
        greek_cols = build_name_format('greek', model, extra_cols, self.default_fill) 
        vol_cols = build_name_format('vol', model, extra_cols, self.default_fill)

        ## Enforce the interval
        enforce_interval(ivl_str)

        ## Assert inputs
        enforce_inputs(type_, model)

        ## Determine aggregation
        agg, database, table = determine_table_agg(ivl_str, type_, greek_names)
        input_params = getattr(self, agg)

        ## Determine the requested columns
        requested_col = determine_requested_columns(self.default_fill, type_, model, greek_names)
        
        data_request = OptionQueryRequestParameter(table_name=table, 
                                                   db_name=database, 
                                                   start_date=start, 
                                                   end_date=end, 
                                                   ticker=self.symbol, 
                                                   exp=self.exp, 
                                                   strike=self.strike,
                                                   right=self.right)
        
        ## Set the parameters for the request to avoid having too many attributes
        data_request.opttick = self.opttick
        data_request.symbol = self.symbol
        data_request.interval= interval
        data_request.type_ = type_
        data_request.input_params = input_params
        data_request.model = model
        data_request.ivl_str = ivl_str
        data_request.ivl_int = ivl_int
        data_request.default_fill = self.default_fill
        data_request.agg = agg
        data_request.requested_col = requested_col + _extra_cols
        data_request.iv_cols = vol_cols
        data_request.greek_cols = greek_cols
        data_request.col_kwargs = col_kwargs = {'underlier_price': 's0', 
             'expiration': 'exp_date', 
             'strike': 'k', 
             'right': 'right', 
             'rf_rate': 'r', 
             'dividend': 'y',
             'put/call': 'right',
             'datetime': 'datetime',}
        self.data_request[self.current_request] = data_request ## save the request for future reference
        
        ## Start by getting query
        init_query(data_request=data_request, db=self.db, query_category='single')

        ## Next, pre process data available in database
        self.__pre_process_data(data_request=data_request)

        ## Before handling missing/incomplete data, we begin save to database
        is_complete = data_request.pre_process['is_complete']
        is_empty = data_request.pre_process['is_empty']
        if is_empty or not is_complete:
            save_thread = Thread(target=save_to_database, args=(data_request, self.print_info), name = "save_to_database", daemon=True)
            save_thread.start()
            self.save_thread[self.current_request] = save_thread
            CENTRAL_SAVE_THREAD[self.current_request] = save_thread

        
        ## Handle missing or incomplete data if any
        self.__handle_incomplete_data(data_request=data_request)
        ## Post process the data
        post_process(data_request=data_request)
        format_final_data(data_request=data_request)
        

            
        return data_request
    
    def get_at_time(self, 
                   date: str | datetime, 
                   type_: str = 'spot',
                   model: str = 'bs',
                   **kwargs) -> pd.DataFrame:
        """
        Get data at a specific time
        params:

        """
        
        if type_ == 'chain':
            return_price = kwargs.get('return_price', False)
            if return_price:
                self.current_request = datetime.now().strftime("%Y%m%d %H:%M:%S")
                request = self.chain_manager.get_at_time(date)
                data = request.post_processed_data.copy()
                self.data_request[self.current_request] = request
                data.columns = data.columns.str.capitalize() 
                data.rename(columns = {'Dte': 'DTE', 'Price': 'Midpoint'}, inplace=True)
                chain = data.pivot_table(
                                    index = ['Expiration', 'DTE', 'Strike'],
                                    columns = ['Right'],
                                    values = ['Midpoint']
                                )
            else: 
                chain = self.Stock.option_chain(date = date)
            return chain
        
        elif type_ in ['spot', 'vol'] + self.greek_names:
            extra_cols = kwargs.get('extra_cols', [])
            return self.get_timeseries(date, date, 
                                       interval = '1d',
                                       type_ = type_,
                                       model = model,
                                       extra_cols=extra_cols).post_processed_data


    def __pre_process_data(self, **kwargs):
        
        data_request = kwargs.get('data_request')
        data = data_request.database_data
        data_request.pre_process = {}

        ## Check if data is empty
        if data.empty:
            ## If data is empty, we will not be able to process it
            data_request.pre_process['is_empty'] = True
        else:
            data_request.pre_process['is_empty'] = False

        ## Check timeseries is complete
        ## Considering we're taking a resample approach, where base intraday data is 5 minutes, and EOD is 1 day
        ## We will only check 5 minutes and 1 day is complete

        start, end = data_request.start_date, data_request.end_date
        date_range = bus_range(start, end, '5Min') if data_request.agg == 'intra' else bus_range(start, end, '1B')

        ## Now we will check if the data is complete
        is_complete = all([(x in pd.DatetimeIndex(data.datetime)) for x in date_range])
        missing_dates = [x for x in date_range if x not in pd.DatetimeIndex(data.datetime)]
        data_request.pre_process['is_complete'] = is_complete
        data_request.missing_dates = missing_dates

        ## Save preprocessed data
        data = data[data_request.requested_col] ## Select only the requested columns
        data.columns = [x.capitalize() for x in data.columns] ## Capitalize the columns
        data.set_index('Datetime', inplace = True) ## Set the index to datetime
        data = data[~data.index.duplicated(keep='first')] ## Remove duplicates
        data_request.pre_processed_data = data

    def __handle_incomplete_data(self, **kwargs):
        data_request = kwargs['data_request']
        is_complete = data_request.pre_process['is_complete']
        is_empty = data_request.pre_process['is_empty']
        start, end, type_ = data_request.start_date, data_request.end_date, data_request.type_
        print(f"Is complete: {is_complete}, Is empty: {is_empty}")

        if is_empty:
            print("Data is empty, Querying all spot data")
            raw_spot_data = self.spot_manager.query_thetadata(start=start, end=end, 
                                                              strike=self.strike, exp=self.exp, 
                                                              right=self.right, bulk=False, 
                                                              data_request=data_request)
            data_request.raw_spot_data = raw_spot_data
            if type_ != 'spot':
                ## Add inputs to raw data, this is necessary for vol calculation
                add_inputs_to_raw(self, data_request=data_request) ## Not formatting yet, this is to utilize joins on datetime
                vol_data = self.vol_manager.calculate_iv(data_request=data_request)
                # data_request.raw_spot_data = pd.concat([data_request.raw_spot_data, vol_data], axis=1)
                if type_ in self.greek_names:
                    greek_data = self.greek_manager.calculate_greeks(type_, data_request = data_request)
            format_raw_spot_data(data_request=data_request)
            


        elif not is_complete:
            print("Data is not complete, Querying missing data")
            start_missing, end_missing = min(data_request.missing_dates), max(data_request.missing_dates)
            raw_spot_data = self.spot_manager.query_thetadata(start=start_missing, end=end_missing, 
                                                              strike=self.strike, exp=self.exp, 
                                                              right=self.right, bulk=False, 
                                                              data_request=data_request)

            data_request.raw_spot_data = raw_spot_data
            if type_ != 'spot':
                add_inputs_to_raw(self, data_request=data_request)
                vol_data = self.vol_manager.calculate_iv(data_request=data_request)
                data_request.raw_spot_data = pd.concat([data_request.raw_spot_data, vol_data], axis=1)
                if type_ in self.greek_names:
                    print(f"Processing Greeks {type_}")
                    greek_data = self.greek_manager.calculate_greeks(type_, data_request = data_request)
                    data_request.raw_spot_data = pd.concat([data_request.raw_spot_data, greek_data], axis=1)
            format_raw_spot_data(data_request=data_request)
        
        else:
            data_request.raw_spot_data = pd.DataFrame()
    


In [81]:
def save_to_database(data_request: OptionQueryRequestParameter, print_info: bool = False):
    """
    Saves the data to the database
    """

    ## This function is using parallel apply to reduce overhead on the current process.
    print(f"Saving data to {data_request.db_name}.{data_request.table_name}") if print_info else None
    
    ## Determine if the data is bulk or not
    if isinstance(data_request, OptionQueryRequestParameter):
        bulk = False
    elif isinstance(data_request, BulkOptionQueryRequestParameter):
        data_request.strike = None
        data_request.right = None
        bulk = True
    else:
        raise ValueError("Expected data_request to be of type OptionQueryRequestParameter or BulkOptionQueryRequestParameter")
    
    db = DatabaseAdapter()
    if len(data_request.missing_dates) == 0:
        print("No missing data, skipping save to database") if print_info else None
        logger.warning("No missing data, skipping save to database")
        return
    start, end = pd.to_datetime(min(data_request.missing_dates)) - relativedelta(months=3), pd.to_datetime(max(data_request.missing_dates)) + relativedelta(months=3)
    print(f"Querying data from {start} to {end}") if print_info else None
    
    ## Start by populating initial data from spot_manager
    spot_manager = SpotDataManager(data_request.symbol)
    spot_sm = spot_manager.query_thetadata(start, end, 
                                                   strike=data_request.strike, exp=data_request.exp, 
                                                   right=data_request.right, bulk=bulk, 
                                                   data_request=data_request)
    
    print("Starting to save data to database") if print_info else None
    if not bulk:
        spot_sm['Strike'] = data_request.strike
        spot_sm['Expiration'] = data_request.exp
        spot_sm['Put/Call'] = data_request.right
        spot_sm['OptionTick'] = data_request.opttick
        spot_sm['Underlier'] = data_request.symbol
        
    else:
        spot_sm.rename(columns = {'k':'strike','exp_date':'expiration', 'Right':'put/call', 'ticker':'Underlier'}, inplace = True)

    spot_sm['Underlier_price'] = data_request.input_params['s0_close']['close']
    spot_sm['RF_rate'] = data_request.input_params['r']
    spot_sm['dividend'] = data_request.input_params['y']
    spot_sm['RF_rate_name'] = data_request.input_params['r_name']
    spot_sm['Datetime'] = spot_sm.index
    spot_sm.columns = [x.lower() for x in spot_sm.columns]
    spot_sm.rename(columns = {'open_interest':'openinterest'}, inplace = True)

    ## Fix for missing data on intraday spot. 
    if data_request.agg == 'intra':
        spot_sm = spot_sm[~spot_sm.underlier_price.isna()]
        if spot_sm.empty:
            logger.warning("Spot data is empty, skipping save to database")
            print("Spot data is empty, skipping save to database")
            return

    ## Add the vol columns
    print("Calculating Vols") if print_info else None
    calc_vol_for_data_parallel(spot_sm, 'close', 'BS_IV', 'bs')
    calc_vol_for_data_parallel(spot_sm, 'midpoint', 'Midpoint_BS_IV', 'bs')
    calc_vol_for_data_parallel(spot_sm, 'weighted_midpoint', 'Weighted_Midpoint_BS_IV', 'bs')
    calc_vol_for_data_parallel(spot_sm, 'closebid', 'bid_bs_iv', 'bs')
    calc_vol_for_data_parallel(spot_sm, 'closeask', 'ask_bs_iv', 'bs')
    

    calc_vol_for_data_parallel(spot_sm, 'close', 'Binomial_IV','binomial')
    calc_vol_for_data_parallel(spot_sm, 'midpoint', 'Midpoint_binomial_IV', 'binomial')
    calc_vol_for_data_parallel(spot_sm, 'weighted_midpoint', 'Weighted_Midpoint_binomial_IV', 'binomial')
    calc_vol_for_data_parallel(spot_sm, 'closebid', 'bid_binomial_iv', 'binomial')
    calc_vol_for_data_parallel(spot_sm, 'closeask', 'ask_binomial_iv', 'binomial')
    spot_sm.columns = spot_sm.columns.str.lower()
    
    

    ## Vol Resolve before Calculating Greeks, this vol is necessary for Greeks
    
    if data_request.agg != 'intra':
        ## Will not be resolving vols for intra data
        print("Resolving Vols") if print_info else None
        resolve_missing_vols_in_data(spot_sm, 
                                    [ 'midpoint_bs_iv',  'midpoint_binomial_iv'], 
                                    ['bs', 'binomial'],
                                    ['midpoint', 'midpoint'],
                                    agg = data_request.agg,)
        

    ## Add the greek columns
    print("Calculating Greeks") if print_info else None
    calc_greeks_for_data_parallel(spot_sm, 'bs', 'bs_iv', '{x}')
    calc_greeks_for_data_parallel(spot_sm, 'bs', 'midpoint_bs_iv', 'midpoint_{x}')
    calc_greeks_for_data_parallel(spot_sm, 'bs', 'weighted_midpoint_bs_iv', 'weighted_midpoint_{x}')

    calc_greeks_for_data_parallel(spot_sm, 'binomial', 'bid_binomial_iv', 'bid_binomial_{x}')
    calc_greeks_for_data_parallel(spot_sm, 'binomial', 'ask_binomial_iv', 'ask_binomial_{x}')
    calc_greeks_for_data_parallel(spot_sm, 'binomial', 'binomial_iv', 'binomial_{x}')
    calc_greeks_for_data_parallel(spot_sm, 'binomial', 'midpoint_binomial_iv', 'midpoint_binomial_{x}')
    spot_sm.columns = spot_sm.columns.str.lower()
    
    ## Add the dollar delta columns
    calc_dollar_delta_from_data(spot_sm, 'delta', 'dollar_delta')
    calc_dollar_delta_from_data(spot_sm, 'midpoint_delta', 'midpoint_dollar_delta')
    calc_dollar_delta_from_data(spot_sm, 'weighted_midpoint_delta', 'weighted_midpoint_dollar_delta')

    ## Add the last updated column
    spot_sm['last_updated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    ## Add the vol resolve columns
    spot_sm['midpoint_bs_vol_resolve'] = 0
    spot_sm['midpoint_binomial_vol_resolve'] = 0

    ## Finally, save the data to the database
    print("Saving data to database") if print_info else None
    data_request.pre_save_to_db_data = spot_sm.copy()
    db.save_to_database(spot_sm, data_request.db_name, data_request.table_name)
    data_request.saved_to_db_data = spot_sm
    
    return spot_sm

In [ ]:
def bulk_one_off_save(
        start: str | datetime,
        end: str | datetime,
        tick: str,
        exp: str,
        print_info: bool = False,
):
    """
    This function is used to save the data to the database without initializing the data manager
    """
    global CENTRAL_SAVE_THREAD
    current_request = datetime.now().strftime("%Y%m%d %H:%M:%S")
    missing_dates = [start, end]
    dummy_request = BulkOptionQueryRequestParameter(
        table_name='temp_options_eod_new',
        db_name='securities_master',
        start_date=start,
        end_date=end,
        ticker=tick,
        exp=exp,
    )
    dummy_request.agg = 'eod'
    dummy_request.start_date = start
    dummy_request.end_date = end
    dummy_request.missing_dates = missing_dates
    lazy_loader = _ManagerLazyLoader(tick)
    lazy_loader.exp = dummy_request.exp
    dummy_request.input_params = lazy_loader.eod
    
    save_thread = Thread(target=save_to_database, args=(dummy_request,print_info), name = "save_to_database_one_off", daemon=True)
    save_thread.start()
    CENTRAL_SAVE_THREAD[current_request] = save_thread


In [56]:
list_contracts('AAPL', '2025-04-21').sort_values('expiration')

,root,expiration,strike,right
1018,AAPL,20250425,235.0,P
1019,AAPL,20250425,235.0,C
1020,AAPL,20250425,128.0,C
1021,AAPL,20250425,160.0,C
1022,AAPL,20250425,160.0,P
...,...,...,...,...
266,AAPL,20271217,430.0,P
267,AAPL,20271217,270.0,C
252,AAPL,20271217,120.0,C
253,AAPL,20271217,120.0,P


In [21]:
# manager = OptionDataManager('AAPL', '2025-09-19', 'C', 225.0) ## EOD
manager = OptionDataManager('AAPL', '2025-09-19', 'P', 270.0) ## Intra
spot_manager = manager.spot_manager

In [22]:
request_ = manager.get_at_time('2024-09-18', 'spot', return_price=False, model = 'binomial', extra_cols = ['ask'])
request_

Using Function to query the database
Is complete: True, Is empty: False
Resampling intra data


,Open,High,Low,Close,Midpoint,Volume,Openinterest,Closeask
Datetime,,,,,,,,
2024-09-18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
vol = manager.get_at_time('2025-03-18', 'vol', return_price=False, model = 'binomial', extra_cols = ['ask'])
vol

Using Function to query the database
Is complete: False, Is empty: True
Data is empty, Querying all spot data
Resampling intra data


,Binomial_iv,Midpoint_binomial_iv,Ask_binomial_iv
Datetime,,,
2025-03-18,0.0,0.0,0.0


In [24]:
chain = manager.get_at_time('2025-04-15', 'chain', return_price=True)
chain

Midpoint         
Right                        C        P
Expiration DTE Strike                  
2025-04-17 2   100.0   102.225    0.005
               110.0    92.225    0.005
               115.0    87.200    0.005
               116.0    86.225    0.005
               117.0    85.225    0.005
...                        ...      ...
2027-12-17 976 420.0     2.770  218.000
               425.0     2.580  223.000
               430.0     2.595  228.000
               435.0     1.500  233.000
               440.0     2.335  238.000

[1213 rows x 2 columns]

## Sequence for Intra Day:

1. Ensure I can get get vol & greeks (Done)
2. Ensure the save data works as expected (Done)
3. Verify it works for both intraday & EOD (Done)
4. Replace vol & greeks with calc vols (Done)
5. Extend to allow new cols (Done)
6. Add chain (Done)
7. Fix Vol Resolve for intra
    - Figure out how exactly to handle this. Because we can't have the thread doing this. It's too much data.

In [25]:
# request = manager.get_timeseries(
#     start='2024-12-17',
#     end='2024-12-18',
#     interval='1h',
#     type_='vol',
#     model='bs',
#     extra_cols=['ask']
# )
# request.post_processed_data


request = manager.get_timeseries(
    start='2025-03-20',
    end='2025-03-30',
    interval='1d',
    type_='vol',
    model='binomial',
    extra_cols=['ask']
)
request.post_processed_data

Using Function to query the database
Is complete: False, Is empty: True
Data is empty, Querying all spot data
Resampling intra data


,Binomial_iv,Midpoint_binomial_iv,Ask_binomial_iv
Datetime,,,
2025-03-20,0.0,0.000100,0.271663
2025-03-21,0.0,0.230543,0.269276
2025-03-24,0.0,0.238067,0.260780
2025-03-25,0.0,0.212444,0.236063
2025-03-26,0.0,0.218778,0.242697
2025-03-27,0.0,0.000100,0.238599
2025-03-28,0.0,0.000100,0.269985


In [26]:
# request.raw_spot_data.set_index('datetime', inplace = True)
# manager.add_inputs_to_raw(data_request=request)
# request.col_kwargs
# request.raw_spot_data


In [27]:
request_current = manager.data_request[manager.current_request]
save_thread = manager.save_thread.get(manager.current_request)
request_current
try:
    print(save_thread.is_alive())
except AttributeError:
    print("No save thread found")

False


In [28]:
class BulkOptionQueryRequestParameter:
    def __init__(self, table_name, db_name, start_date=None, end_date=None, ticker=None, exp=None, strikes=None):
        self.db_name = db_name
        self.table_name = table_name
        self.start_date = start_date
        self.end_date = end_date
        self.ticker = ticker
        self.exp = exp
        self.strikes  = strikes
        self.opttick = None
        self.symbol = ticker
        self.query = None
        self.y = None
        self.vol = None
        self.spot = None
        self.interval = None
        self.type_ = None

## BULK DATA MANAGER

In [83]:
from typing import TYPE_CHECKING, List, Tuple
class BulkOptionDataManager(_ManagerLazyLoader):
    @log_time(time_logger)
    def __init__(self,
                symbol: str = None,
                exp: str | datetime = None,
                default_fill: str = 'midpoint',
                **kwargs) -> None:
        """
        Returns an object for querying data

        Params:
        symbol: Underlier symbol
        exp: expiration
        right: Put(P) or Call (C)
        strike: Option Strike
        default_fill: How to fill zero values for close. 'midpoint' or 'weighted_midpoint'
        opttick: Option ticker, if provided, will ignore symbol, exp, right, strike and be initialized with the string
        """

        super().__init__(symbol)
        if default_fill not in ['midpoint', 'weighted_midpoint', None]:
            raise ValueError("Expected default_fill to be one of: 'midpoint', 'weighted_midpoint', None ")
        
        assert all([symbol, exp,]), "symbol, exp, are required"
        self.exp = exp
        self.symbol = symbol

        self.default_fill = default_fill
        self.db = DatabaseAdapter()
        self.data_request = {}
        self.save_thread = {}
        self.current_request =None
        self.spot_manager = SpotDataManager(self.symbol)
        self.vol_manager = VolDataManager(self.symbol)
        self.greek_manager = GreeksDataManager(self.symbol)
        self.chain_manager = ChainDataManager(self.symbol)
        self.greek_names = PRICING_CONFIG["AVAILABLE_GREEKS"] + ['greek', 'greeks']
        self.print_info = kwargs.get('print_info', False)

        ## Prefer to use dicts to avoid having too many attributes
        self._eod = {}

    def get_timeseries(self, 
                       start: str | datetime, 
                       end: str | datetime,
                       interval: str = '1d',
                       type_: str = 'spot',
                       strikes_right: List[Tuple] = [],
                       model: str = 'bs',
                       extra_cols: list = []) -> pd.DataFrame:
        """
        Query the timeseries data from ThetaData API or SQL Database.
        Params:
        start: Start date for the query
        end: End date for the query
        interval: Interval for the query. Options are: h, d, w, M, q, y
        type_: Type of data to query. Options are: spot, vol, greeks, greek, attribution, scenario
        model: Model to use for the query. Options are: bs, binomial
        extra_cols: Extra columns to include in the query. Options are: ask, bid, open
        strikes_right: List of tuples containing the strike and right for the options. Eg: [(250, 'C'), (225, 'P')]
        """
        
        if not strikes_right:
            raise ValueError("Strikes cannot be empty")
        
        assert isinstance(strikes_right, list), f"Strikes has to be type list, recieved {type(strikes_right)}"
        assert all([isinstance(x, tuple) for x in strikes_right]), f"Strikes has to be type list of tuples, recieved {type(strikes_right)}"
        
        ## Organize inputs
        start = pd.to_datetime(start)
        end = pd.to_datetime(end)
        ivl_str, ivl_int = extract_numeric_value(interval)
        greek_names = self.greek_names
        self.current_request = datetime.now().strftime("%Y%m%d %H:%M:%S")
        _extra_cols = handle_extra_cols(extra_cols, type_, model)
        greek_cols = build_name_format('greek', model, extra_cols, self.default_fill)
        vol_cols = build_name_format('vol', model, extra_cols, self.default_fill)


        ## Enforce the interval
        enforce_interval(ivl_str)

        ## Assert inputs
        enforce_inputs(type_, model)

        ## Determine aggregation
        agg, database, table = determine_table_agg(ivl_str, type_, greek_names)
        input_params = getattr(self, agg)

        ## Determine the requested columns
        requested_col = determine_requested_columns(self.default_fill, type_, model, greek_names)

        data_request = BulkOptionQueryRequestParameter(table_name=table,
                                                         db_name=database, 
                                                         start_date=start, 
                                                         end_date=end, 
                                                         ticker=self.symbol, 
                                                         exp=self.exp, 
                                                         strikes=strikes_right)
        
        ## Set the parameters for the request to avoid having too many attributes
        data_request.symbol = self.symbol
        data_request.interval= interval
        data_request.type_ = type_
        data_request.input_params = input_params
        data_request.model = model
        data_request.ivl_str = ivl_str
        data_request.ivl_int = ivl_int
        data_request.default_fill = self.default_fill
        data_request.agg = agg
        data_request.requested_col = requested_col + _extra_cols + ['optiontick']
        data_request.iv_cols = vol_cols
        data_request.greek_cols = greek_cols
        data_request.col_kwargs = col_kwargs = {'underlier_price': 's0', 
             'expiration': 'exp_date', 
             'strike': 'k', 
             'right': 'right', 
             'rf_rate': 'r', 
             'dividend': 'y',
             'put/call': 'right',
             'datetime': 'datetime',}
        self.data_request[self.current_request] = data_request ## save the request for future reference

        ## Start by getting query
        init_query(data_request=data_request, db=self.db, query_category='bulk')
        ## Next, pre process data available in database
        self.__pre_process_data(data_request=data_request)

        # ## Before handling missing/incomplete data, we begin save to database
        # is_complete = data_request.pre_process['is_complete']
        # is_empty = data_request.pre_process['is_empty']
        # if is_empty or not is_complete:
        #     save_thread = Thread(target=save_to_database, args=(data_request, self.print_info), name = "save_to_database", daemon=True)
        #     save_thread.start()
        #     self.save_thread[self.current_request] = save_thread

        ## Handle missing or incomplete data if any
        self.__handle_incomplete_data(data_request=data_request)

        ## Post process the data
        post_process(data_request=data_request, bulk = True)
        
        ## Format the data
        format_final_data(data_request=data_request, bulk = True)
        return data_request 
 
    ## Make a function
    def __handle_incomplete_data(self, **kwargs):
        data_request = kwargs['data_request']
        is_complete = data_request.pre_process['is_complete']
        is_empty = data_request.pre_process['is_empty']
        start, end, type_ = data_request.start_date, data_request.end_date, data_request.type_
        print(f"Is complete: {is_complete}, Is empty: {is_empty}")

        if is_empty:
            print("Data is empty, Querying all spot data")
            raw_spot_data = self.spot_manager.query_thetadata(start=start, end=end, 
                                                              strike=None, exp=self.exp, 
                                                              right=None, bulk=True, 
                                                              data_request=data_request)
            data_request.raw_spot_data = raw_spot_data
            if type_ != 'spot':
                ## Add inputs to raw data, this is necessary for vol calculation
                add_inputs_to_raw(self, data_request=data_request, bulk = True) ## Not formatting yet, this is to utilize joins on datetime
                vol_data = self.vol_manager.calculate_iv(data_request=data_request)
                # data_request.raw_spot_data = pd.concat([data_request.raw_spot_data, vol_data], axis=1)
                if type_ in self.greek_names:
                    greek_data = self.greek_manager.calculate_greeks(type_, data_request = data_request)
            format_raw_spot_data(data_request=data_request)
            


        elif not is_complete:
            print("Data is not complete, Querying missing data")
            start_missing, end_missing = min(data_request.missing_dates), max(data_request.missing_dates)
            raw_spot_data = self.spot_manager.query_thetadata(start=start_missing, end=end_missing, 
                                                              strike=None, exp=self.exp, 
                                                              right=None, bulk=True, 
                                                              data_request=data_request)

            # raw_spot_data['Datetime'] = pd.to_datetime(raw_spot_data['Datetime'])
            data_request.raw_spot_data = raw_spot_data
            if type_ != 'spot':
                add_inputs_to_raw(self, data_request=data_request, bulk = True)
                vol_data = self.vol_manager.calculate_iv(data_request=data_request)
                data_request.raw_spot_data = pd.concat([data_request.raw_spot_data, vol_data], axis=1)
                if type_ in self.greek_names:
                    print(f"Processing Greeks {type_}")
                    greek_data = self.greek_manager.calculate_greeks(type_, data_request = data_request)
                    data_request.raw_spot_data = pd.concat([data_request.raw_spot_data, greek_data], axis=1)
            format_raw_spot_data(data_request=data_request)
        
        else:
            data_request.raw_spot_data = pd.DataFrame()

    def __pre_process_data(self, **kwargs):
        data_request = kwargs.get('data_request')
        data = data_request.database_data
        data_request.pre_process = {}

        ## Check timeseries is complete
        ## Considering we're taking a resample approach, where base intraday data is 5 minutes, and EOD is 1 day
        ## We will only check 5 minutes and 1 day is complete

        start, end = data_request.start_date, data_request.end_date
        date_range = bus_range(start, end, '5Min') if data_request.agg == 'intra' else bus_range(start, end, '1B')

        ## Transform the data to Opttick as columns, close as values, datetime as index
        transformed = data_request.database_data.pivot_table(
            index = ['datetime'],
            columns = ['optiontick'],
            values = ['close']
        )
        transformed.columns = transformed.columns.droplevel(0)


        ## First Completeness check: Do we have all OptTicks?
        first_check = all(x in transformed.columns for x in data_request.opttick)
        ## This will fill missing option ticks with NaN
        transformed[[x for x in data_request.opttick if x not in transformed.columns.get_level_values(0)]] = np.nan


        ## Second Completeness check: Do we have all dates?
        missing_dates_second_check = [x for x in date_range if x not in (transformed.index)]
        second_check = all(x in pd.DatetimeIndex(transformed.index) for x in date_range)
        transformed = transformed.reindex(date_range, fill_value=np.nan) ## This will fill the missing dates with NaN

        ## Third Completeness check: If we have all Ticks, do all ticks have all dates? 
        ## This will exclude dates from second check
        if not first_check:
            third_check = False
        else:
            ## Check if all dates are present for all ticks
            third_check = not transformed.isna().any().any()

        complete_check = first_check and second_check and third_check

        ## Is empty Check
        is_empty = data_request.database_data.empty

        # Missing Dates: This will be tiered
        if not first_check: ## This means all dates are missing for one name, we have to query all dates
            missing_dates = date_range
        else: ## This means some dates are missing for some names.
            missing_dates = transformed[transformed.isna().any(axis = 1)].index.to_list()

        ## Save preprocessed data
        data_request.pre_process['is_complete'] = complete_check
        data_request.pre_process['is_empty'] = is_empty
        data_request.missing_dates = missing_dates

        data = data[data_request.requested_col] ## Select only the requested columns
        data.columns = [x.capitalize() for x in data.columns] ## Capitalize the columns
        data.set_index([ 'Optiontick', 'Datetime',], inplace = True) ## Set the index to datetime
        data = data[~data.index.duplicated(keep='first')] ## Remove duplicates
        data.sort_index(inplace = True)
        data_request.pre_processed_data = data

    @staticmethod
    def one_off_save(start:str, 
                     end:str, 
                     tick:str, 
                     exp:str, 
                     print_info:bool = False):
        """
        This function is used to save the data to the database without initializing the data manager
        """
        bulk_one_off_save(start, end, tick, exp, print_info)


In [90]:
BulkOptionDataManager.one_off_save( start='2025-03-27',
    end='2025-04-05',
    tick='AAPL',
    exp='2025-05-02',
    print_info=True
)

Saving data to securities_master.temp_options_eod_new
Querying data from 2024-12-27 00:00:00 to 2025-07-05 00:00:00


Starting to save data to database
Calculating Vols
Resolving Vols
2025-04-21 18:06:34 trade.models.utils ERROR: AAPL20250502C110, 2025-03-24 could not fit SVI model. Returning zero
2025-04-21 18:08:01 trade.models.utils ERROR: AAPL20250502C140, 2025-04-21 could not fit SVI model. Returning zero
2025-04-21 18:08:12 trade.models.utils ERROR: AAPL20250502C145, 2025-03-24 could not fit SVI model. Returning zero
2025-04-21 18:08:24 trade.models.utils ERROR: AAPL20250502C145, 2025-04-21 could not fit SVI model. Returning zero
2025-04-21 18:08:34 trade.models.utils ERROR: AAPL20250502C150, 2025-03-24 could not fit SVI model. Returning zero
2025-04-21 18:08:45 trade.models.utils ERROR: AAPL20250502C160, 2025-03-24 could not fit SVI model. Returning zero
2025-04-21 18:09:56 trade.models.utils ERROR: AAPL20250502P280, 2025-04-14 could not fit SVI model. Returning zero
2025-04-21 18:10:09 trade.models.utils ERROR: AAPL20250502P285, 2025-04-14 could not fit SVI model. Returning zero
2025-04-21 18:

Exception in thread save_to_database_one_off:
Traceback (most recent call last):
  File "/Users/chiemelienwanisobi/miniconda3/envs/openbb/lib/python3.11/site-packages/pandas/core/indexing.py", line 1714, in _get_list_axis
    return self.obj._take_with_is_copy(key, axis=axis)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/chiemelienwanisobi/miniconda3/envs/openbb/lib/python3.11/site-packages/pandas/core/generic.py", line 4153, in _take_with_is_copy
    result = self.take(indices=indices, axis=axis)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/chiemelienwanisobi/miniconda3/envs/openbb/lib/python3.11/site-packages/pandas/core/generic.py", line 4133, in take
    new_data = self._mgr.take(
               ^^^^^^^^^^^^^^^
  File "/Users/chiemelienwanisobi/miniconda3/envs/openbb/lib/python3.11/site-packages/pandas/core/internals/managers.py", line 891, in take
    indexer = maybe_convert_indices(indexer, n, verify=verify)
              ^^^^^^^^^^^^^

In [58]:
# bulk_manager = BulkOptionDataManager('AAPL', '2025-09-19', default_fill = 'midpoint')
bulk_manager = BulkOptionDataManager('AAPL', '2025-04-25', default_fill = 'midpoint')
bulk_manager.print_info = True

In [64]:
# request = manager.get_timeseries(
#     start='2024-12-17',
#     end='2024-12-18',
#     interval='1h',
#     type_='vol',
#     model='bs',
#     extra_cols=['ask']
# )
# request.post_processed_data


request = bulk_manager.get_timeseries(
    start='2025-03-27',
    end='2025-04-05',
    interval='1d',
    type_='spot',
    model='binomial',
    extra_cols=['ask'],
    strikes_right = [(280.0, 'C'), (270.0, 'P'), (225.0, 'C'), (270.0, 'C'), (250.0, 'P') ]
)
request.post_processed_data

Using Function to query the database
Is complete: False, Is empty: True
Data is empty, Querying all spot data
Resampling intra data
2025-04-21 16:39:50 dbase.DataAPI.ThetaData CRITICAL: `datetime_col_name` not provided for multi index resample, setting to `Datetime`


Open   High    Low  Close  Midpoint  Volume  \
Optiontick       Datetime                                                   
AAPL20250425C225 2025-03-27   4.90   6.56   4.56   6.05     6.150     674   
                 2025-03-28   5.00   5.94   3.48   3.48     3.500     520   
                 2025-03-31   3.48   6.15   3.20   5.30     5.225    1130   
                 2025-04-01   4.45   5.75   3.91   5.43     5.425     737   
                 2025-04-02   4.70   6.10   4.60   5.71     5.650     629   
                 2025-04-03   1.06   1.20   0.77   1.11     1.110    3084   
                 2025-04-04   0.60   1.08   0.44   0.50     0.540    2547   
AAPL20250425C270 2025-03-27   0.06   0.06   0.06   0.06     0.040       6   
                 2025-03-28   0.06   0.06   0.03   0.04     0.050      26   
                 2025-03-31   0.02   0.02   0.02   0.02     0.040       1   
                 2025-04-01   0.03   0.03   0.03   0.03     0.035       2   
                 2025-04-02   0.00   0.00   0.00   0.00     0.025       0   
                 2025-04-03   0.06   0.06   0.01   0.01     0.010       5   
                 2025-04-04   0.00   0.00   0.00   0.00     0.300       0   
AAPL20250425C280 2025-03-27   0.00   0.00   0.00   0.00     0.130       0   
                 2025-03-28   0.00   0.00   0.00   0.00     0.035       0   
                 2025-03-31   0.02   0.02   0.02   0.02     0.010       1   
                 2025-04-01   0.00   0.00   0.00   0.00     0.010       0   
                 2025-04-02   0.02   0.03   0.02   0.03     0.025       5   
                 2025-04-03   0.02   0.03   0.02   0.03     0.015      10   
                 2025-04-04   0.00   0.00   0.00   0.00     0.015       0   
AAPL20250425P250 2025-03-27  25.60  26.30  25.45  26.30    26.175     145   
                 2025-03-28  28.30  32.45  28.30  32.40    32.175     112   
                 2025-03-31   0.00   0.00   0.00   0.00    28.600       0   
                 2025-04-01   0.00   0.00   0.00   0.00    26.875       0   
                 2025-04-02  25.75  25.75  25.75  25.75    26.650       1   
                 2025-04-03  47.15  47.15  46.95  46.95    47.075       4   
                 2025-04-04  58.82  58.82  58.82  58.82    61.500      10   
AAPL20250425P270 2025-03-27  46.21  46.21  46.10  46.10    45.650       2   
                 2025-03-28   0.00   0.00   0.00   0.00    51.575       0   
                 2025-03-31   0.00   0.00   0.00   0.00    48.625       0   
                 2025-04-01   0.00   0.00   0.00   0.00    46.925       0   
                 2025-04-02   0.00   0.00   0.00   0.00    46.100       0   
                 2025-04-03   0.00   0.00   0.00   0.00    67.050       0   
                 2025-04-04   0.00   0.00   0.00   0.00    81.500       0   

                             Openinterest  Closeask  
Optiontick       Datetime                            
AAPL20250425C225 2025-03-27         10820      6.30  
                 2025-03-28         10962      3.55  
                 2025-03-31         11011      5.35  
                 2025-04-01         11280      5.50  
                 2025-04-02         11340      5.80  
                 2025-04-03         11352      1.15  
                 2025-04-04         11864      0.64  
AAPL20250425C270 2025-03-27           160      0.07  
                 2025-03-28           166      0.06  
                 2025-03-31           156      0.06  
                 2025-04-01           157      0.04  
                 2025-04-02           159      0.03  
                 2025-04-03           159      0.02  
                 2025-04-04           164      0.60  
AAPL20250425C280 2025-03-27            90      0.25  
                 2025-03-28            90      0.05  
                 2025-03-31            90      0.02  
                 2025-04-01            91      0.02  
                 2025-04-02            91      0.03  
                 2025-04-03            86     

In [63]:
current_request = bulk_manager.data_request[bulk_manager.current_request]  
# add_inputs_to_raw(bulk_manager, data_request=current_request) 
current_request.post_processed_data

Open   High    Low  Close  Midpoint  Volume  \
Optiontick       Datetime                                                   
AAPL20250425C225 2025-03-27   4.90   6.56   4.56   6.05     6.150     674   
                 2025-03-28   5.00   5.94   3.48   3.48     3.500     520   
                 2025-03-31   3.48   6.15   3.20   5.30     5.225    1130   
                 2025-04-01   4.45   5.75   3.91   5.43     5.425     737   
                 2025-04-02   4.70   6.10   4.60   5.71     5.650     629   
                 2025-04-03   1.06   1.20   0.77   1.11     1.110    3084   
                 2025-04-04   0.60   1.08   0.44   0.50     0.540    2547   
AAPL20250425C270 2025-03-27   0.06   0.06   0.06   0.06     0.040       6   
                 2025-03-28   0.06   0.06   0.03   0.04     0.050      26   
                 2025-03-31   0.02   0.02   0.02   0.02     0.040       1   
                 2025-04-01   0.03   0.03   0.03   0.03     0.035       2   
                 2025-04-02   0.00   0.00   0.00   0.00     0.025       0   
                 2025-04-03   0.06   0.06   0.01   0.01     0.010       5   
                 2025-04-04   0.00   0.00   0.00   0.00     0.300       0   
AAPL20250425C280 2025-03-27   0.00   0.00   0.00   0.00     0.130       0   
                 2025-03-28   0.00   0.00   0.00   0.00     0.035       0   
                 2025-03-31   0.02   0.02   0.02   0.02     0.010       1   
                 2025-04-01   0.00   0.00   0.00   0.00     0.010       0   
                 2025-04-02   0.02   0.03   0.02   0.03     0.025       5   
                 2025-04-03   0.02   0.03   0.02   0.03     0.015      10   
                 2025-04-04   0.00   0.00   0.00   0.00     0.015       0   
AAPL20250425P250 2025-03-27  25.60  26.30  25.45  26.30    26.175     145   
                 2025-03-28  28.30  32.45  28.30  32.40    32.175     112   
                 2025-03-31   0.00   0.00   0.00   0.00    28.600       0   
                 2025-04-01   0.00   0.00   0.00   0.00    26.875       0   
                 2025-04-02  25.75  25.75  25.75  25.75    26.650       1   
                 2025-04-03  47.15  47.15  46.95  46.95    47.075       4   
                 2025-04-04  58.82  58.82  58.82  58.82    61.500      10   
AAPL20250425P270 2025-03-27  46.21  46.21  46.10  46.10    45.650       2   
                 2025-03-28   0.00   0.00   0.00   0.00    51.575       0   
                 2025-03-31   0.00   0.00   0.00   0.00    48.625       0   
                 2025-04-01   0.00   0.00   0.00   0.00    46.925       0   
                 2025-04-02   0.00   0.00   0.00   0.00    46.100       0   
                 2025-04-03   0.00   0.00   0.00   0.00    67.050       0   
                 2025-04-04   0.00   0.00   0.00   0.00    81.500       0   

                             Openinterest  Closeask  
Optiontick       Datetime                            
AAPL20250425C225 2025-03-27         10820      6.30  
                 2025-03-28         10962      3.55  
                 2025-03-31         11011      5.35  
                 2025-04-01         11280      5.50  
                 2025-04-02         11340      5.80  
                 2025-04-03         11352      1.15  
                 2025-04-04         11864      0.64  
AAPL20250425C270 2025-03-27           160      0.07  
                 2025-03-28           166      0.06  
                 2025-03-31           156      0.06  
                 2025-04-01           157      0.04  
                 2025-04-02           159      0.03  
                 2025-04-03           159      0.02  
                 2025-04-04           164      0.60  
AAPL20250425C280 2025-03-27            90      0.25  
                 2025-03-28            90      0.05  
                 2025-03-31            90      0.02  
                 2025-04-01            91      0.02  
                 2025-04-02            91      0.03  
                 2025-04-03            86     

In [73]:
def bulk_one_off_save(
        start: str | datetime,
        end: str | datetime,
        tick: str,
        exp: str,
        print_info: bool = False,
):
    """
    This function is used to save the data to the database without initializing the data manager
    """
    global CENTRAL_SAVE_THREAD
    current_request = datetime.now().strftime("%Y%m%d %H:%M:%S")
    missing_dates = [start, end]
    dummy_request = BulkOptionQueryRequestParameter(
        table_name='temp_options_eod_new',
        db_name='securities_master',
        start_date=start,
        end_date=end,
        ticker=tick,
        exp=exp,
    )
    dummy_request.agg = 'eod'
    dummy_request.start_date = start
    dummy_request.end_date = end
    dummy_request.missing_dates = missing_dates
    lazy_loader = _ManagerLazyLoader(tick)
    lazy_loader.exp = dummy_request.exp
    dummy_request.input_params = lazy_loader.eod
    
    save_thread = Thread(target=save_to_database, args=(dummy_request,print_info), name = "save_to_database_one_off", daemon=True)
    save_thread.start()
    CENTRAL_SAVE_THREAD[current_request] = save_thread


bulk_one_off_save(
    start='2025-03-27',
    end='2025-04-05',
    tick='AAPL',
    exp='2025-04-25',
)

In [101]:
CENTRAL_SAVE_THREAD

{'20250421 16:29:53': <Thread(save_to_database, stopped daemon 123145699987456)>,
 '20250421 16:30:00': <Thread(save_to_database, stopped daemon 123145699987456)>,
 '20250421 16:55:48': <Thread(save_to_database_one_off, stopped daemon 123145767145472)>,
 '20250421 16:59:43': <Thread(save_to_database_one_off, stopped daemon 123145767145472)>,
 '20250421 17:00:27': <Thread(save_to_database_one_off, stopped daemon 123145767145472)>,
 '20250421 17:22:50': <Thread(save_to_database_one_off, started daemon 123145834303488)>,
 '20250421 17:38:53': <Thread(save_chain_data, stopped daemon 123145851092992)>}

In [833]:

start, end = current_request.start_date, current_request.end_date
date_range = bus_range(start, end, '5Min') if current_request.agg == 'intra' else bus_range(start, end, '1B')

## Transform the data to Opttick as columns, close as values, datetime as index
transformed = current_request.database_data.pivot_table(
    index = ['datetime'],
    columns = ['optiontick'],
    values = ['close']
)
transformed.columns = transformed.columns.droplevel(0)


## First Completeness check: Do we have all OptTicks?
first_check = all(x in transformed.columns for x in current_request.opttick)
## This will fill missing option ticks with NaN
transformed[[x for x in current_request.opttick if x not in transformed.columns.get_level_values(0)]] = np.nan


## Second Completeness check: Do we have all dates?
missing_dates_second_check = [x for x in date_range if x not in (transformed.index)]
second_check = all(x in pd.DatetimeIndex(transformed.index) for x in date_range)
transformed = transformed.reindex(date_range, fill_value=np.nan) ## This will fill the missing dates with NaN

## Third Completeness check: If we have all Ticks, do all ticks have all dates? 
## This will exclude dates from second check
if not first_check:
    third_check = False
else:
    ## Check if all dates are present for all ticks
    third_check = not transformed.isna().any().any()

complete_check = first_check and second_check and third_check

## Is empty Check
is_empty = current_request.database_data.empty

# Missing Dates: This will be tiered
if not first_check: ## This means all dates are missing for one name, we have to query all dates
    missing_dates = date_range
else: ## This means some dates are missing for some names.
    missing_dates = transformed[transformed.isna().any(axis = 1)].index.to_list()

print(f"First Check: {first_check}, Second Check: {second_check}, Third Check: {third_check}, Complete Check: {complete_check}, Is Empty: {is_empty}")

First Check: True, Second Check: True, Third Check: True, Complete Check: True, Is Empty: False


In [367]:
## Query Bulk EOD
start = '2024-01-01'
end = '2024-12-31'
bulk = retrieve_bulk_eod(
    symbol = 'META',
    exp = '2025-09-19',
    start_date = start,
    end_date = end,
)

## Add Option Tick
bulk_eod = bulk.reset_index()
tick_col = ['Root', 'Right', 'Expiration', 'Strike']
bulk_eod['OptionTick'] = parallel_apply(bulk_eod[tick_col], generate_option_tick_new, pool = POOL_ENABLED)


## Query Bulk Open Interest
bulk_oi = retrieve_bulk_open_interest(
    symbol = 'META',
    exp = '2025-09-19',
    start_date = start,
    end_date = end,
)
## Add Option Tick
bulk_oi['OptionTick'] = parallel_apply(bulk_oi[tick_col], generate_option_tick_new, pool = POOL_ENABLED)
## Add EOD Timestamp
bulk_oi['Datetime'] = add_eod_timestamp(pd.DatetimeIndex(bulk_oi['Datetime']))


bulk_merged = bulk_eod.merge(bulk_oi[['Datetime','OptionTick', 'Open_interest']], on = ['Datetime', 'OptionTick'], how = 'left')
bulk_merged[bulk_merged.Open_interest.isna()]

,Datetime,Root,Strike,Expiration,Right,Open,High,Low,Close,Volume,Bid_size,CloseBid,Ask_size,CloseAsk,Midpoint,Weighted_midpoint,OptionTick,Open_interest
60480,2024-07-10 16:00:00,META,705.0,2025-09-19,C,0.00,0.00,0.00,0.00,0,21,41.25,10,43.30,42.275,41.911290,META20250919C705,NaN
60602,2024-07-10 16:00:00,META,705.0,2025-09-19,P,0.00,0.00,0.00,0.00,0,49,182.20,11,186.15,184.175,182.924167,META20250919P705,NaN
61228,2024-07-10 16:00:00,META,715.0,2025-09-19,C,0.00,0.00,0.00,0.00,0,21,39.10,8,41.00,40.050,39.624138,META20250919C715,NaN
61350,2024-07-10 16:00:00,META,715.0,2025-09-19,P,0.00,0.00,0.00,0.00,0,34,190.55,11,194.30,192.425,191.466667,META20250919P715,NaN
61976,2024-07-10 16:00:00,META,725.0,2025-09-19,C,39.00,39.00,38.95,39.00,6,14,37.45,16,38.70,38.075,38.116667,META20250919C725,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80868,2024-12-09 16:00:00,META,1230.0,2025-09-19,P,0.00,0.00,0.00,0.00,0,56,613.10,50,620.35,616.725,616.519811,META20250919P1230,NaN
80884,2024-12-09 16:00:00,META,1240.0,2025-09-19,C,0.00,0.00,0.00,0.00,0,5,2.22,5,2.37,2.295,2.295000,META20250919C1240,NaN
80900,2024-12-09 16:00:00,META,1240.0,2025-09-19,P,0.00,0.00,0.00,0.00,0,56,623.10,50,630.40,626.750,626.543396,META20250919P1240,NaN
80916,2024-12-09 16:00:00,META,1250.0,2025-09-19,C,2.36,2.36,2.36,2.36,1,9,2.12,1,2.57,2.345,2.165000,META20250919C1250,NaN


In [280]:
request.agg = 'eod'
spot_manager = SpotDataManager('META')
bulk_eod_from_manager = spot_manager.query_thetadata(
    data_request = request,
    start = '2024-01-01',
    end = '2024-12-31',
    exp = '2025-09-19',
    right = 'C',
    bulk = True
)
bulk_eod_from_manager

,Datetime,Root,Strike,Expiration,Right,Open,High,Low,Close,Volume,Bid_size,CloseBid,Ask_size,CloseAsk,Midpoint,Weighted_midpoint,OptionTick,Open_interest
0,2024-01-02 16:00:00,META,5.0,2025-09-19,C,0.0,0.0,0.0,0.0,0,18,339.90,28,343.65,341.775,342.182609,META20250919C5,0.0
1,2024-01-03 16:00:00,META,5.0,2025-09-19,C,0.0,0.0,0.0,0.0,0,10,337.50,10,342.50,340.000,340.000000,META20250919C5,0.0
2,2024-01-04 16:00:00,META,5.0,2025-09-19,C,0.0,0.0,0.0,0.0,0,14,340.70,14,344.65,342.675,342.675000,META20250919C5,0.0
3,2024-01-05 16:00:00,META,5.0,2025-09-19,C,0.0,0.0,0.0,0.0,0,28,345.45,28,349.40,347.425,347.425000,META20250919C5,0.0
4,2024-01-08 16:00:00,META,5.0,2025-09-19,C,0.0,0.0,0.0,0.0,0,14,352.15,14,356.05,354.100,354.100000,META20250919C5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80943,2024-12-24 16:00:00,META,1250.0,2025-09-19,P,0.0,0.0,0.0,0.0,0,1,640.50,1,646.50,643.500,643.500000,META20250919P1250,0.0
80944,2024-12-26 16:00:00,META,1250.0,2025-09-19,P,0.0,0.0,0.0,0.0,0,11,644.95,12,648.90,646.925,647.010870,META20250919P1250,0.0
80945,2024-12-27 16:00:00,META,1250.0,2025-09-19,P,0.0,0.0,0.0,0.0,0,12,648.35,11,652.30,650.325,650.239130,META20250919P1250,0.0
80946,2024-12-30 16:00:00,META,1250.0,2025-09-19,P,0.0,0.0,0.0,0.0,0,16,656.90,17,660.95,658.925,658.986364,META20250919P1250,0.0


'eod'

In [488]:
db = DatabaseAdapter()
columns = db.query_database('securities_master', 'temp_options_eod_new', "SELECT * FROM securities_master.temp_options_eod_new WHERE 1=0").columns
columns = [x.lower() for x in columns]

## Database Save

### References

In [ ]:
spot_sm = spot_manager.query_thetadata('2023-02-03', '2023-02-10', strike=220.0, exp='2024-09-20', right='C', bulk=False, data_request=request)
spot_sm['Strike'] = manager.strike
spot_sm['Expiration'] = manager.exp
spot_sm['Put/Call'] = manager.right
spot_sm['Underlier_price'] = request.input_params['s0_close']['close']
spot_sm['RF_rate'] = request.input_params['r']
spot_sm['dividend'] = request.input_params['y']
spot_sm['OptionTick'] = manager.opttick
spot_sm['Underlier'] = manager.symbol
spot_sm['RF_rate_name'] = request.input_params['r_name']
spot_sm['Datetime'] = spot_sm.index
spot_sm.columns = [x.lower() for x in spot_sm.columns]
spot_sm.rename(columns = {'open_interest':'openinterest'}, inplace = True)
[x for x in columns if x not in spot_sm.columns]
# spot_sm

### Vol Calcs

In [25]:

calc_vol_for_data(spot_sm, 'close', 'BS_IV', 'bs')
calc_vol_for_data(spot_sm, 'midpoint', 'Midpoint_BS_IV', 'bs')
calc_vol_for_data(spot_sm, 'weighted_midpoint', 'Weighted_Midpoint_BS_IV', 'bs')
calc_vol_for_data(spot_sm, 'closebid', 'bid_bs_iv', 'bs')
calc_vol_for_data(spot_sm, 'closeask', 'ask_bs_iv', 'bs')

calc_vol_for_data(spot_sm, 'close', 'Binomial_IV','binomial')
calc_vol_for_data(spot_sm, 'midpoint', 'Midpoint_binomial_IV', 'binomial')
calc_vol_for_data(spot_sm, 'weighted_midpoint', 'Weighted_Midpoint_binomial_IV', 'binomial')
calc_vol_for_data(spot_sm, 'closebid', 'bid_binomial_iv', 'binomial')
calc_vol_for_data(spot_sm, 'closeask', 'ask_binomial_iv', 'binomial')


In [23]:
spot_sm.columns = spot_sm.columns.str.lower()
[x for x in columns if x not in spot_sm.columns]


['delta',
 'gamma',
 'vega',
 'theta',
 'rho',
 'vanna',
 'volga',
 'dollar_delta',
 'midpoint_delta',
 'midpoint_gamma',
 'midpoint_vega',
 'midpoint_theta',
 'midpoint_rho',
 'midpoint_vanna',
 'midpoint_volga',
 'midpoint_dollar_delta',
 'weighted_midpoint_delta',
 'weighted_midpoint_gamma',
 'weighted_midpoint_vega',
 'weighted_midpoint_theta',
 'weighted_midpoint_rho',
 'weighted_midpoint_vanna',
 'weighted_midpoint_volga',
 'weighted_midpoint_dollar_delta',
 'midpoint_binomial_gamma',
 'midpoint_binomial_vega',
 'midpoint_binomial_delta',
 'midpoint_binomial_rho',
 'midpoint_binomial_vanna',
 'midpoint_binomial_volga',
 'midpoint_binomial_theta',
 'last_updated',
 'binomial_delta',
 'binomial_gamma',
 'binomial_vega',
 'binomial_volga',
 'binomial_vanna',
 'binomial_rho',
 'binomial_theta',
 'midpoint_bs_vol_resolve',
 'midpoint_binomial_vol_resolve',
 'ask_binomial_delta',
 'ask_binomial_gamma',
 'ask_binomial_vega',
 'ask_binomial_rho',
 'ask_binomial_theta',
 'ask_binomial_van

### Greeks

In [26]:
calc_greeks_for_data(spot_sm, 'bs', 'bs_iv', '{x}')
calc_greeks_for_data(spot_sm, 'bs', 'midpoint_bs_iv', 'midpoint_{x}')
calc_greeks_for_data(spot_sm, 'bs', 'weighted_midpoint_bs_iv', 'weighted_midpoint_{x}')

calc_greeks_for_data(spot_sm, 'binomial', 'bid_binomial_iv', 'bid_binomial_{x}')
calc_greeks_for_data(spot_sm, 'binomial', 'ask_binomial_iv', 'ask_binomial_{x}')
calc_greeks_for_data(spot_sm, 'binomial', 'binomial_iv', 'binomial_{x}')
calc_greeks_for_data(spot_sm, 'binomial', 'midpoint_binomial_iv', 'midpoint_binomial_{x}')
spot_sm.columns


Index(['open', 'high', 'low', 'close', 'volume', 'bid_size', 'closebid',
       'ask_size', 'closeask', 'midpoint', 'weighted_midpoint', 'openinterest',
       'strike', 'expiration', 'put/call', 'underlier_price', 'rf_rate',
       'dividend', 'optiontick', 'underlier', 'rf_rate_name', 'datetime',
       'bs_iv', 'midpoint_bs_iv', 'weighted_midpoint_bs_iv', 'bid_bs_iv',
       'ask_bs_iv', 'binomial_iv', 'midpoint_binomial_iv',
       'weighted_midpoint_binomial_iv', 'bid_binomial_iv', 'ask_binomial_iv',
       'Delta', 'Gamma', 'Vega', 'Theta', 'Rho', 'Vanna', 'Volga',
       'midpoint_Delta', 'midpoint_Gamma', 'midpoint_Vega', 'midpoint_Theta',
       'midpoint_Rho', 'midpoint_Vanna', 'midpoint_Volga',
       'weighted_midpoint_Delta', 'weighted_midpoint_Gamma',
       'weighted_midpoint_Vega', 'weighted_midpoint_Theta',
       'weighted_midpoint_Rho', 'weighted_midpoint_Vanna',
       'weighted_midpoint_Volga', 'bid_binomial_Delta', 'bid_binomial_Gamma',
       'bid_binomial_Vega',

In [27]:
spot_sm
spot_sm.columns = spot_sm.columns.str.lower()
[x for x in columns if x not in spot_sm.columns]

['dollar_delta',
 'midpoint_dollar_delta',
 'weighted_midpoint_dollar_delta',
 'last_updated',
 'midpoint_bs_vol_resolve',
 'midpoint_binomial_vol_resolve']

In [28]:
calc_dollar_delta_from_data(spot_sm, 'delta', 'dollar_delta')
calc_dollar_delta_from_data(spot_sm, 'midpoint_delta', 'midpoint_dollar_delta')
calc_dollar_delta_from_data(spot_sm, 'weighted_midpoint_delta', 'weighted_midpoint_dollar_delta')

,open,high,low,close,volume,bid_size,closebid,ask_size,closeask,midpoint,weighted_midpoint,openinterest,strike,expiration,put/call,underlier_price,rf_rate,dividend,optiontick,underlier,rf_rate_name,datetime,bs_iv,midpoint_bs_iv,weighted_midpoint_bs_iv,bid_bs_iv,ask_bs_iv,binomial_iv,midpoint_binomial_iv,weighted_midpoint_binomial_iv,bid_binomial_iv,ask_binomial_iv,delta,gamma,vega,theta,rho,vanna,volga,midpoint_delta,midpoint_gamma,midpoint_vega,midpoint_theta,midpoint_rho,midpoint_vanna,midpoint_volga,weighted_midpoint_delta,weighted_midpoint_gamma,weighted_midpoint_vega,weighted_midpoint_theta,weighted_midpoint_rho,weighted_midpoint_vanna,weighted_midpoint_volga,bid_binomial_delta,bid_binomial_gamma,bid_binomial_vega,bid_binomial_theta,bid_binomial_rho,bid_binomial_vanna,bid_binomial_volga,ask_binomial_delta,ask_binomial_gamma,ask_binomial_vega,ask_binomial_theta,ask_binomial_rho,ask_binomial_vanna,ask_binomial_volga,binomial_delta,binomial_gamma,binomial_vega,binomial_theta,binomial_rho,binomial_vanna,binomial_volga,midpoint_binomial_delta,midpoint_binomial_gamma,midpoint_binomial_vega,midpoint_binomial_theta,midpoint_binomial_rho,midpoint_binomial_vanna,midpoint_binomial_volga,dollar_delta,midpoint_dollar_delta,weighted_midpoint_dollar_delta
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-02-03,5.35,5.35,5.35,5.35,1,356,4.60,143,5.50,5.050,4.857916,126,225.0,2024-09-20,C,154.500000,0.04523,0.005890,AAPL20240920C225,AAPL,^IRX,2023-02-03,0.257526,0.252241,0.248803,0.244116,0.260133,0.256354,0.251083,0.247690,0.243112,0.258977,0.214354,0.005721,0.572577,-0.015309,0.452498,0.926439,143.288841,0.207662,0.005733,0.562061,-0.014757,0.440554,0.929907,148.088487,0.203248,0.005739,0.554956,-0.014396,0.432583,0.931856,151.260767,0.195835,0.005744,0.542715,-0.013796,0.419031,0.934479,156.591731,0.216172,0.005716,0.575381,-0.015460,0.455713,0.925396,141.988426,0.212880,0.005724,0.570287,-0.015187,0.449882,0.927254,144.344526,0.206180,0.005736,0.559691,-0.014636,0.437887,0.930592,149.152772,33.117713,32.083747,31.401856
2023-02-06,4.95,4.95,4.85,4.85,6,126,4.65,159,5.00,4.825,4.845263,126,225.0,2024-09-20,C,151.729996,0.04533,0.004548,AAPL20240920C225,AAPL,^IRX,2023-02-06,0.258361,0.257898,0.258273,0.254638,0.261119,0.257615,0.257170,0.257531,0.254049,0.260280,0.201097,0.005611,0.540696,-0.014616,0.416097,0.903942,145.352654,0.200514,0.005612,0.539757,-0.014568,0.415070,0.904100,145.741815,0.200986,0.005612,0.540518,-0.014607,0.415903,0.903972,145.426300,0.195635,0.005613,0.531800,-0.014164,0.406423,0.905231,148.998685,0.203506,0.005610,0.544554,-0.014817,0.420329,0.903234,143.743226,0.200157,0.005612,0.539180,-0.014538,0.414440,0.904195,145.980408,0.199595,0.005612,0.538271,-0.014491,0.413448,0.904340,146.355531,30.512406,30.424001,30.495677
2023-02-07,5.05,5.05,5.05,5.05,1,135,5.20,140,5.80,5.500,5.505455,127,225.0,2024-09-20,C,154.649994,0.04570,0.004462,AAPL20240920C225,AAPL,^IRX,2023-02-07,0.251048,0.258927,0.259022,0.253699,0.264068,0.250110,0.257993,0.258089,0.252746,0.263210,0.208762,0.005798,0.563006,-0.014984,0.440850,0.935349,148.207753,0.218742,0.005776,0.578500,-0.015818,0.458526,0.929665,141.019470,0.218860,0.005776,0.578679,-0.015828,0.458732,0.929590,140.934929,0.210934,0.005794,0.566435,-0.015164,0.444729,0.934224,146.639797,0.224060,0.005760,0.586482,-0.016267,0.467784,0.926128,137.212817,0.207556,0.005800,0.561088,-0.014884,0.438689,0.935946,149.078786,0.217573,0.005779,0.576719,-0.015719,0.456474,0.930397,141.859324,32.285053,33.828508,33.846727
2023-02-08,0.00,0.00,0.00,0.00,0,432,4.50,478,5.55,5.025,5.051538,126,225.0,2024-09-20,C,151.919998,0.04585,0.004542,AAPL20240920C225,AAPL,^IRX,2023-02-08,0.000000,0.261004,0.261488,0.251243,0.270434,0.000000,0.260136,0.260607,0.250763,0.269409,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.205469,0.005629,0.547433,-0.015021,0.423204,0.904255,142.635135,0.206073,0.005628,0.548385,-0.015072,0.424257,0.904043,1

In [29]:
spot_sm
spot_sm.columns = spot_sm.columns.str.lower()
[x for x in columns if x not in spot_sm.columns]

['last_updated', 'midpoint_bs_vol_resolve', 'midpoint_binomial_vol_resolve']

In [30]:
spot_sm['last_updated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

### Vol Resolve & Last Updated

In [31]:

[x for x in columns if x not in spot_sm.columns]

['midpoint_bs_vol_resolve', 'midpoint_binomial_vol_resolve']

In [33]:
resolve_missing_vols_in_data(spot_sm, 
                             [ 'midpoint_bs_iv',  'midpoint_binomial_iv'], 
                             ['bs', 'binomial'],
                             ['midpoint', 'midpoint'])

,open,high,low,close,volume,bid_size,closebid,ask_size,closeask,midpoint,weighted_midpoint,openinterest,strike,expiration,put/call,underlier_price,rf_rate,dividend,optiontick,underlier,rf_rate_name,datetime,bs_iv,midpoint_bs_iv,weighted_midpoint_bs_iv,bid_bs_iv,ask_bs_iv,binomial_iv,midpoint_binomial_iv,weighted_midpoint_binomial_iv,bid_binomial_iv,ask_binomial_iv,delta,gamma,vega,theta,rho,vanna,volga,midpoint_delta,midpoint_gamma,midpoint_vega,midpoint_theta,midpoint_rho,midpoint_vanna,midpoint_volga,weighted_midpoint_delta,weighted_midpoint_gamma,weighted_midpoint_vega,weighted_midpoint_theta,weighted_midpoint_rho,weighted_midpoint_vanna,weighted_midpoint_volga,bid_binomial_delta,bid_binomial_gamma,bid_binomial_vega,bid_binomial_theta,bid_binomial_rho,bid_binomial_vanna,bid_binomial_volga,ask_binomial_delta,ask_binomial_gamma,ask_binomial_vega,ask_binomial_theta,ask_binomial_rho,ask_binomial_vanna,ask_binomial_volga,binomial_delta,binomial_gamma,binomial_vega,binomial_theta,binomial_rho,binomial_vanna,binomial_volga,midpoint_binomial_delta,midpoint_binomial_gamma,midpoint_binomial_vega,midpoint_binomial_theta,midpoint_binomial_rho,midpoint_binomial_vanna,midpoint_binomial_volga,dollar_delta,midpoint_dollar_delta,weighted_midpoint_dollar_delta,last_updated
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-02-03,5.35,5.35,5.35,5.35,1,356,4.60,143,5.50,5.050,4.857916,126,225.0,2024-09-20,C,154.500000,0.04523,0.005890,AAPL20240920C225,AAPL,^IRX,2023-02-03,0.257526,0.252241,0.248803,0.244116,0.260133,0.256354,0.251083,0.247690,0.243112,0.258977,0.214354,0.005721,0.572577,-0.015309,0.452498,0.926439,143.288841,0.207662,0.005733,0.562061,-0.014757,0.440554,0.929907,148.088487,0.203248,0.005739,0.554956,-0.014396,0.432583,0.931856,151.260767,0.195835,0.005744,0.542715,-0.013796,0.419031,0.934479,156.591731,0.216172,0.005716,0.575381,-0.015460,0.455713,0.925396,141.988426,0.212880,0.005724,0.570287,-0.015187,0.449882,0.927254,144.344526,0.206180,0.005736,0.559691,-0.014636,0.437887,0.930592,149.152772,33.117713,32.083747,31.401856,2025-04-11 20:10:06
2023-02-06,4.95,4.95,4.85,4.85,6,126,4.65,159,5.00,4.825,4.845263,126,225.0,2024-09-20,C,151.729996,0.04533,0.004548,AAPL20240920C225,AAPL,^IRX,2023-02-06,0.258361,0.257898,0.258273,0.254638,0.261119,0.257615,0.257170,0.257531,0.254049,0.260280,0.201097,0.005611,0.540696,-0.014616,0.416097,0.903942,145.352654,0.200514,0.005612,0.539757,-0.014568,0.415070,0.904100,145.741815,0.200986,0.005612,0.540518,-0.014607,0.415903,0.903972,145.426300,0.195635,0.005613,0.531800,-0.014164,0.406423,0.905231,148.998685,0.203506,0.005610,0.544554,-0.014817,0.420329,0.903234,143.743226,0.200157,0.005612,0.539180,-0.014538,0.414440,0.904195,145.980408,0.199595,0.005612,0.538271,-0.014491,0.413448,0.904340,146.355531,30.512406,30.424001,30.495677,2025-04-11 20:10:06
2023-02-07,5.05,5.05,5.05,5.05,1,135,5.20,140,5.80,5.500,5.505455,127,225.0,2024-09-20,C,154.649994,0.04570,0.004462,AAPL20240920C225,AAPL,^IRX,2023-02-07,0.251048,0.258927,0.259022,0.253699,0.264068,0.250110,0.257993,0.258089,0.252746,0.263210,0.208762,0.005798,0.563006,-0.014984,0.440850,0.935349,148.207753,0.218742,0.005776,0.578500,-0.015818,0.458526,0.929665,141.019470,0.218860,0.005776,0.578679,-0.015828,0.458732,0.929590,140.934929,0.210934,0.005794,0.566435,-0.015164,0.444729,0.934224,146.639797,0.224060,0.005760,0.586482,-0.016267,0.467784,0.926128,137.212817,0.207556,0.005800,0.561088,-0.014884,0.438689,0.935946,149.078786,0.217573,0.005779,0.576719,-0.015719,0.456474,0.930397,141.859324,32.285053,33.828508,33.846727,2025-04-11 20:10:06
2023-02-08,0.00,0.00,0.00,0.00,0,432,4.50,478,5.55,5.025,5.051538,126,225.0,2024-09-20,C,151.919998,0.04585,0.004542,AAPL20240920C225,AAPL,^IRX,2023-02-08,0.000000,0.261004,0.261488,0.251243,0.270434,0.000000,0.260136,0.260607,0.250763,0.269409,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.205469,0.005629,0.547433,-0.015021,0.423204,0.

In [34]:

spot_sm['midpoint_bs_vol_resolve'] = 0
spot_sm['midpoint_binomial_vol_resolve'] = 0
[x for x in columns if x not in spot_sm.columns]

[]

In [35]:
db.save_to_database(spot_sm, 'securities_master', 'temp_options_eod_new')


In [36]:
columns
print("Columns in Save Data not in Database")
print([x for x in spot_sm.columns if x not in columns])

print("Columns in Database not in Save Data")
print([x for x in columns if x not in spot_sm.columns])

Columns in Save Data not in Database
[]
Columns in Database not in Save Data
[]


In [53]:
runThreads(save_to_database, [[request_v2], [db],[manager]], block=False, thread_name_prefix='save_to_database')

Running in non-blocking mode with 1 threads
Saving data to securities_master.temp_options_eod_new
Inputs: data_request: <__main__.OptionQueryRequestParameter object at 0x14144a250>, db: <dbase.database.SQLHelpers.DatabaseAdapter object at 0x14249c650>, manager: <__main__.OptionDataManager object at 0x14027fb50>


<generator object Executor.map.<locals>.result_iterator at 0x140fe3140>

2025-04-11 20:21:43 trade.models.VolSurface ERROR: Expiration date is the same as the datetime. Returning 0
2025-04-11 20:22:47 trade.models.VolSurface ERROR: Expiration date is the same as the datetime. Returning 0


In [52]:
runThreads?

Signature:
runThreads(
    func,
    OrderedInputs: list[list],
    run_type: str = 'map',
    block=True,
    thread_name_prefix='',
) -> list
Docstring:
Run multithreading on a given function.

params:
--------
func: Function to run in multiple threads.
OrderedInputs: List of inputs to pass to the function. Example:
               [[input1, input1, input1], [input2, input2, input2], [input3, input3, input3]]

run_type: Type of multithreading execution. Default is 'map'.
block: Boolean flag to indicate if results should be returned. Default is True.
            If False, the function will return a list of futures. 
            Note: Returning a list of futures is non blocking.
thread_name_prefix: Prefix for the thread names. Default is empty string.

returns:
--------
List of results from the threading function.
File:      ~/cloned_repos/QuantTools/trade/helpers/threads.py
Type:      function

In [767]:
save_to_database?

Signature:
save_to_database(
    data_request: __main__.OptionQueryRequestParameter,
    db: dbase.database.SQLHelpers.DatabaseAdapter,
    manager: __main__.OptionDataManager,
)
Docstring: Saves the data to the database
File:      /var/folders/j0/80hkbygd4lb27h9mw76gqzpw0000gn/T/ipykernel_53121/576773820.py
Type:      function

In [755]:
save_data = save_to_database(request_v2, db, manager)

In [762]:
db.save_to_database(save_data, 'securities_master', 'temp_options_eod_new')


(253, 87)
(253, 87)


## TO-DO

- Implement IntraDay
- Implement Chain in timeseries.
    - This will query database and get necessary data
- Implement OHLC Bulk
    - With gendata & save data
- Implement Scenario & Attribution Database
- Binomial Model: Greeks & Vol queries
- Extra Columns input
    - Use calc_vol_for_data to calculate extra columns
- Add vol resolve to vol manager
- Revisist SVI 
    - Using Binomial Vols
    - Is it using Forward?d

In [13]:
from queue import Queue, Full

In [10]:

class SaveManager:

    ## Setting initial variables
    MAX_QUEUE_SIZE = 100
    WORKER_COUNT = 4
    _queue = Queue(maxsize=MAX_QUEUE_SIZE)
    _threads = []
    _started = False

    @classmethod
    def _worker(cls):
        while True: ## Starts the forever loop
            pack = cls._queue.get() ## Once the queue is empty, it will block until a new item is added
            request, save_func = pack
            if request is None:
                break
            try:
                save_func(request) ## This is the task being being handled in the worker thread
            except Exception as e:
                logger.error(f"[SaveWorker] Error processing save: {e}")
            finally:
                cls._queue.task_done() ## Good practice to mark the task as done

    
    @classmethod
    def start_workers(cls):
        """
        This class starts the worker threads for saving data to the database.
        It is more or less a list of threads in a forever loop.
        """
        if cls._started:
            return
        for _ in range(cls.WORKER_COUNT):
            t = Thread(target=cls._worker, daemon=True) ## The _worker function will run in a separate thread. Looping forever
            t.start()
            cls._threads.append(t)
        cls._started = True
        logger.info(f"[SaveManager] Started {cls.WORKER_COUNT} save workers.")

    @classmethod
    def enqueue(cls, data_request, save_func):
        """
        Enqueue a save request to the queue.
        """
        try:
            print(f"[SaveManager] Enqueueing save request for {data_request.symbol} on {data_request}")
            cls._queue.put((data_request, save_func), block=False)  # Will raise Full if over limit
        except Full:
            logger.warning(f"[SaveManager] Save queue full (max {cls.MAX_QUEUE_SIZE}). Task ignored.")
        
    @classmethod
    def status(cls):
        return {
            "pending_tasks": cls._queue.qsize(),
            "max_queue_size": cls._queue.maxsize,
            "active_threads": sum(t.is_alive() for t in cls._threads),
            "total_threads": len(cls._threads),
        }


(1, 2)

In [12]:
from queue import Queue

q = Queue()

q.put("task")
q.get()
q.task_done()  #❌ missing

q.join()  # 🧨 will block forever!
